# 🧬 CAFA 6: Complete Exploratory Data Analysis
- Author: Dr.Hany Ghazal (PhD)
- Competition: CAFA 6 - Protein Function Prediction
- Last Updated: October 2025

In [ ]:
!pip install biopython obonet

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
import networkx as nx
from typing import Dict, List, Tuple, Set
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# First, we need to write functions for data loading

In [ ]:
data_dir = "/kaggle/input/cafa-6-protein-function-prediction/"

In [ ]:
# File paths
train_seq_file = f"{data_dir}Train/train_sequences.fasta"
test_seq_file = f"{data_dir}Test/testsuperset.fasta"
terms_file = f"{data_dir}Train/train_terms.tsv"
taxonomy_file = f"{data_dir}Train/train_taxonomy.tsv"
ia_file = f"{data_dir}IA.tsv"
obo_file = f"{data_dir}Train/go-basic.obo"

In [ ]:
# ============================================
# 1. DATA LOADING FUNCTIONS
# ============================================

def load_fasta_sequences(fasta_file: str) -> Dict[str, str]:
    """
    Load protein sequences from FASTA file.
    
    Args:
        fasta_file: Path to FASTA file
        
    Returns:
        Dictionary mapping protein ID to sequence
    """
    sequences = {}
    
    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            # Extract UniProt ID from header (format: sp|ID|NAME)
            header_parts = record.id.split('|')
            if len(header_parts) >= 2:
                protein_id = header_parts[1]
            else:
                protein_id = record.id
            
            sequences[protein_id] = str(record.seq)
            
        print(f"Loaded {len(sequences):,} sequences from {fasta_file}")
        
    except FileNotFoundError:
        print(f"Error: File {fasta_file} not found")
        # Return sample data for demo
        sequences = {
            'P9WHI7': 'MTKPTQVLVRLEQVMAEGDLRTLVPLLKAGQYRDDVTFGGPTVHPGVRVEGHTL',
            'P04637': 'MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFT'
        }
        print("Using sample data for demonstration")
        
    return sequences


def load_go_annotations(terms_file: str) -> pd.DataFrame:
    """
    Load GO term annotations.
    
    Args:
        terms_file: Path to train_terms.tsv
        
    Returns:
        DataFrame with columns: protein_id, go_term, ontology
    """
    try:
        df = pd.read_csv(terms_file, sep='\t', 
                        names=['protein_id', 'go_term', 'ontology'])
        print(f"Loaded {len(df):,} GO annotations for {df['protein_id'].nunique():,} proteins")
        
    except FileNotFoundError:
        print(f"Error: File {terms_file} not found")
        # Sample data for demo
        df = pd.DataFrame({
            'protein_id': ['P9WHI7', 'P9WHI7', 'P04637', 'P04637'],
            'go_term': ['GO:0009274', 'GO:0071944', 'GO:1990837', 'GO:0031625'],
            'ontology': ['BPO', 'CCO', 'MFO', 'MFO']
        })
        print("Using sample data for demonstration")
        
    return df


def load_taxonomy(taxonomy_file: str) -> pd.DataFrame:
    """
    Load taxonomy information.
    
    Args:
        taxonomy_file: Path to train_taxonomy.tsv
        
    Returns:
        DataFrame with protein_id and taxon_id
    """
    try:
        df = pd.read_csv(taxonomy_file, sep='\t', 
                        names=['protein_id', 'taxon_id'])
        print(f"Loaded taxonomy for {len(df):,} proteins across {df['taxon_id'].nunique()} species")
        
    except FileNotFoundError:
        print(f"Error: File {taxonomy_file} not found")
        # Sample data
        df = pd.DataFrame({
            'protein_id': ['P9WHI7', 'P04637'],
            'taxon_id': [1773, 9606]
        })
        print("Using sample data for demonstration")
        
    return df


def load_information_accretion(ia_file: str) -> pd.DataFrame:
    """
    Load information accretion weights.
    
    Args:
        ia_file: Path to IA.tsv
        
    Returns:
        DataFrame with go_term and ia_weight
    """
    try:
        df = pd.read_csv(ia_file, sep='\t', names=['go_term', 'ia_weight'])
        print(f"Loaded IA weights for {len(df):,} GO terms")
        
    except FileNotFoundError:
        print(f"Error: File {ia_file} not found")
        # Sample data
        df = pd.DataFrame({
            'go_term': ['GO:0009274', 'GO:0071944', 'GO:1990837', 'GO:0031625'],
            'ia_weight': [0.5, 0.3, 0.8, 0.6]
        })
        print("Using sample data for demonstration")
        
    return df


def parse_obo_file(obo_file: str) -> nx.DiGraph:
    """
    Parse GO ontology OBO file and create directed graph.
    
    Args:
        obo_file: Path to go-basic.obo
        
    Returns:
        NetworkX directed graph of GO ontology
    """
    try:
        import obonet
        graph = obonet.read_obo(obo_file)
        print(f"Loaded GO ontology: {graph.number_of_nodes():,} terms, {graph.number_of_edges():,} relationships")
        return graph
        
    except (FileNotFoundError, ImportError):
        print(f"Error loading {obo_file} or obonet not installed")
        print("Creating sample graph for demonstration")
        
        # Create sample GO graph
        G = nx.DiGraph()
        sample_terms = [
            ('GO:0008150', 'biological_process'),
            ('GO:0009987', 'cellular process'),
            ('GO:0009274', 'peptidoglycan-based cell wall'),
            ('GO:0003674', 'molecular_function'),
            ('GO:0005575', 'cellular_component'),
        ]
        
        for term_id, name in sample_terms:
            G.add_node(term_id, name=name)
        
        # Add some edges
        G.add_edge('GO:0008150', 'GO:0009987')
        G.add_edge('GO:0009987', 'GO:0009274')
        
        return G

In [ ]:
# 1. Load all data
print("\n" + "="*60)
print("1. LOADING DATA")
print("="*60)

train_sequences = load_fasta_sequences(train_seq_file)
test_sequences = load_fasta_sequences(test_seq_file)
annotations = load_go_annotations(terms_file)
taxonomy = load_taxonomy(taxonomy_file)
ia_weights = load_information_accretion(ia_file)
go_graph = parse_obo_file(obo_file)

# Sequence Analysis Functions

In [ ]:
def analyze_sequence_statistics(sequences: Dict[str, str]) -> pd.DataFrame:
    """
    Calculate basic statistics for protein sequences.
    
    Args:
        sequences: Dictionary mapping protein_id to sequence
        
    Returns:
        DataFrame with sequence statistics
    """
    stats = []
    
    for protein_id, seq in sequences.items():
        stats.append({
            'protein_id': protein_id,
            'length': len(seq),
            'molecular_weight': calculate_molecular_weight(seq),
            'num_A': seq.count('A'),
            'num_C': seq.count('C'),
            'num_D': seq.count('D'),
            'num_E': seq.count('E'),
            'num_F': seq.count('F'),
            'num_G': seq.count('G'),
            'num_H': seq.count('H'),
            'num_I': seq.count('I'),
            'num_K': seq.count('K'),
            'num_L': seq.count('L'),
            'num_M': seq.count('M'),
            'num_N': seq.count('N'),
            'num_P': seq.count('P'),
            'num_Q': seq.count('Q'),
            'num_R': seq.count('R'),
            'num_S': seq.count('S'),
            'num_T': seq.count('T'),
            'num_V': seq.count('V'),
            'num_W': seq.count('W'),
            'num_Y': seq.count('Y'),
        })
    
    df = pd.DataFrame(stats)
    
    print("\nSequence Statistics Summary:")
    print(f"Total proteins: {len(df):,}")
    print(f"Length - Mean: {df['length'].mean():.1f}, Median: {df['length'].median():.1f}")
    print(f"Length - Min: {df['length'].min()}, Max: {df['length'].max()}")
    print(f"Molecular Weight - Mean: {df['molecular_weight'].mean():.1f} Da")
    
    return df

def calculate_molecular_weight(sequence: str) -> float:
    """Calculate approximate molecular weight of protein sequence."""
    aa_weights = {
        'A': 89.1, 'C': 121.2, 'D': 133.1, 'E': 147.1, 'F': 165.2,
        'G': 75.1, 'H': 155.2, 'I': 131.2, 'K': 146.2, 'L': 131.2,
        'M': 149.2, 'N': 132.1, 'P': 115.1, 'Q': 146.2, 'R': 174.2,
        'S': 105.1, 'T': 119.1, 'V': 117.1, 'W': 204.2, 'Y': 181.2
    }
    
    weight = sum(aa_weights.get(aa, 0) for aa in sequence)
    return weight - (len(sequence) - 1) * 18.015  # Subtract water molecules


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from typing import Dict, List, Tuple
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Set global plotting style
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12


# ============================================================================
# 1. SEQUENCE VISUALIZATIONS
# ============================================================================

def plot_sequence_length_distribution(sequences: Dict[str, str], 
                                      title: str = "Sequence Length Distribution",
                                      save_path: str = None):
    """
    Plot distribution of protein sequence lengths.
    
    Args:
        sequences: Dictionary mapping protein_id to sequence
        title: Plot title
        save_path: Optional path to save figure
    """
    lengths = [len(seq) for seq in sequences.values()]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histogram
    axes[0].hist(lengths, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].axvline(np.mean(lengths), color='red', linestyle='--', 
                    linewidth=2, label=f'Mean: {np.mean(lengths):.0f}')
    axes[0].axvline(np.median(lengths), color='orange', linestyle='--', 
                    linewidth=2, label=f'Median: {np.median(lengths):.0f}')
    axes[0].set_xlabel('Sequence Length (amino acids)', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title(title, fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Box plot
    axes[1].boxplot(lengths, vert=True, patch_artist=True,
                   boxprops=dict(facecolor='lightblue', alpha=0.7),
                   medianprops=dict(color='red', linewidth=2))
    axes[1].set_ylabel('Sequence Length (amino acids)', fontsize=12)
    axes[1].set_title('Box Plot', fontsize=14, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print statistics
    print(f"\n{'='*60}")
    print(f"SEQUENCE LENGTH STATISTICS")
    print(f"{'='*60}")
    print(f"Total sequences: {len(lengths):,}")
    print(f"Mean length: {np.mean(lengths):.2f} AA")
    print(f"Median length: {np.median(lengths):.2f} AA")
    print(f"Std deviation: {np.std(lengths):.2f} AA")
    print(f"Min length: {min(lengths)} AA")
    print(f"Max length: {max(lengths):,} AA")
    print(f"25th percentile: {np.percentile(lengths, 25):.0f} AA")
    print(f"75th percentile: {np.percentile(lengths, 75):.0f} AA")
    print(f"{'='*60}\n")

In [ ]:
analyze_sequence_statistics(train_sequences)

In [ ]:
plot_sequence_length_distribution(train_sequences, save_path="/kaggle/working/train_sequence_length_distribution.png")

In [ ]:
analyze_sequence_statistics(test_sequences)

In [ ]:
plot_sequence_length_distribution(test_sequences, save_path="/kaggle/working/test_sequence_length_distribution.png")

In [ ]:
def analyze_amino_acid_composition(sequences: Dict[str, str]) -> pd.DataFrame:
    """
    Calculate amino acid composition statistics.
    
    Returns:
        DataFrame with AA frequencies
    """
    total_counts = Counter()
    
    for seq in sequences.values():
        total_counts.update(seq)
    
    total_aa = sum(total_counts.values())
    
    composition = pd.DataFrame([
        {'amino_acid': aa, 'count': count, 'frequency': count/total_aa}
        for aa, count in sorted(total_counts.items())
    ])
    
    print("\nAmino Acid Composition (Top 10):")
    print(composition.nlargest(10, 'frequency')[['amino_acid', 'frequency']])
    
    return composition




In [ ]:
analyze_amino_acid_composition(train_sequences)

In [ ]:
analyze_amino_acid_composition(test_sequences)

In [ ]:
def plot_amino_acid_composition(sequences: Dict[str, str], 
                                top_n: int = 20,
                                save_path: str = None):
    """
    Plot amino acid composition across all sequences.
    
    Args:
        sequences: Dictionary mapping protein_id to sequence
        top_n: Number of amino acids to display (default 20 for all)
        save_path: Optional path to save figure
    """
    # Count amino acids
    total_counts = Counter()
    for seq in sequences.values():
        total_counts.update(seq)
    
    total_aa = sum(total_counts.values())
    
    # Create DataFrame
    aa_data = pd.DataFrame([
        {'AA': aa, 'Count': count, 'Frequency': count/total_aa*100}
        for aa, count in sorted(total_counts.items())
    ]).sort_values('Frequency', ascending=False).head(top_n)
    
    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Bar chart
    colors = plt.cm.viridis(np.linspace(0, 1, len(aa_data)))
    axes[0].bar(aa_data['AA'], aa_data['Frequency'], 
               color=colors, alpha=0.8, edgecolor='black')
    axes[0].set_xlabel('Amino Acid', fontsize=12)
    axes[0].set_ylabel('Frequency (%)', fontsize=12)
    axes[0].set_title('Amino Acid Composition', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add percentage labels on bars
    for i, (aa, freq) in enumerate(zip(aa_data['AA'], aa_data['Frequency'])):
        axes[0].text(i, freq + 0.2, f'{freq:.1f}%', 
                    ha='center', va='bottom', fontsize=8)
    
    # Pie chart (top 10)
    top10 = aa_data.head(10)
    axes[1].pie(top10['Frequency'], labels=top10['AA'], autopct='%1.1f%%',
               colors=colors[:10], startangle=90)
    axes[1].set_title('Top 10 Amino Acids', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print statistics
    print(f"\n{'='*60}")
    print(f"AMINO ACID COMPOSITION")
    print(f"{'='*60}")
    print(f"Total amino acids: {total_aa:,}")
    print(f"\nTop 10 Most Frequent:")
    for i, row in aa_data.head(10).iterrows():
        print(f"  {row['AA']}: {row['Count']:>12,} ({row['Frequency']:>5.2f}%)")
    print(f"{'='*60}\n")


In [ ]:
plot_amino_acid_composition(train_sequences, save_path="/kaggle/working/train_amino_acid_composition.png")

In [ ]:
plot_amino_acid_composition(test_sequences, save_path="/kaggle/working/test_amino_acid_composition.png")

In [ ]:
def plot_sequence_comparison(train_seqs: Dict[str, str], 
                            test_seqs: Dict[str, str],
                            save_path: str = None):
    """
    Compare train and test sequence distributions.
    
    Args:
        train_seqs: Training sequences
        test_seqs: Test sequences
        save_path: Optional path to save figure
    """
    train_lengths = [len(seq) for seq in train_seqs.values()]
    test_lengths = [len(seq) for seq in test_seqs.values()]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Overlapping histograms
    axes[0, 0].hist(train_lengths, bins=50, color='blue', alpha=0.5, 
                   label=f'Train (n={len(train_lengths):,})', edgecolor='black')
    axes[0, 0].hist(test_lengths, bins=50, color='red', alpha=0.5, 
                   label=f'Test (n={len(test_lengths):,})', edgecolor='black')
    axes[0, 0].set_xlabel('Sequence Length', fontsize=12)
    axes[0, 0].set_ylabel('Frequency', fontsize=12)
    axes[0, 0].set_title('Length Distribution Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Box plots
    axes[0, 1].boxplot([train_lengths, test_lengths], 
                      labels=['Train', 'Test'],
                      patch_artist=True,
                      boxprops=dict(alpha=0.7))
    axes[0, 1].set_ylabel('Sequence Length', fontsize=12)
    axes[0, 1].set_title('Box Plot Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].grid(alpha=0.3)
    
    # Violin plots
    data_to_plot = [train_lengths, test_lengths]
    parts = axes[1, 0].violinplot(data_to_plot, positions=[1, 2], 
                                  showmeans=True, showmedians=True)
    axes[1, 0].set_xticks([1, 2])
    axes[1, 0].set_xticklabels(['Train', 'Test'])
    axes[1, 0].set_ylabel('Sequence Length', fontsize=12)
    axes[1, 0].set_title('Violin Plot Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].grid(alpha=0.3)
    
    # Statistical comparison table
    axes[1, 1].axis('off')
    stats_data = [
        ['Metric', 'Train', 'Test', 'Difference'],
        ['Count', f'{len(train_lengths):,}', f'{len(test_lengths):,}', '-'],
        ['Mean', f'{np.mean(train_lengths):.1f}', f'{np.mean(test_lengths):.1f}', 
         f'{np.mean(train_lengths) - np.mean(test_lengths):+.1f}'],
        ['Median', f'{np.median(train_lengths):.1f}', f'{np.median(test_lengths):.1f}',
         f'{np.median(train_lengths) - np.median(test_lengths):+.1f}'],
        ['Std Dev', f'{np.std(train_lengths):.1f}', f'{np.std(test_lengths):.1f}',
         f'{np.std(train_lengths) - np.std(test_lengths):+.1f}'],
        ['Min', f'{min(train_lengths)}', f'{min(test_lengths)}', '-'],
        ['Max', f'{max(train_lengths):,}', f'{max(test_lengths):,}', '-'],
    ]
    
    table = axes[1, 1].table(cellText=stats_data, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Color header row
    for i in range(4):
        table[(0, i)].set_facecolor('#4ECDC4')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    axes[1, 1].set_title('Statistical Comparison', fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
plot_sequence_comparison(train_sequences, test_sequences, save_path="/kaggle/working/sequence_comparison.png")

# Target Label Analysis Functions (For Annotations)

In [ ]:
def plot_labels_per_protein(annotations: pd.DataFrame,
                           max_labels: int = 150,
                           save_path: str = None):
    """
    Plot distribution of labels per protein.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology columns
        max_labels: Maximum number of labels to display on x-axis
        save_path: Optional path to save figure
    """
    labels_per_protein = annotations.groupby('protein_id').size()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Histogram
    axes[0, 0].hist(labels_per_protein, bins=50, color='coral', 
                   alpha=0.7, edgecolor='black')
    axes[0, 0].axvline(labels_per_protein.mean(), color='red', linestyle='--',
                      linewidth=2, label=f'Mean: {labels_per_protein.mean():.1f}')
    axes[0, 0].axvline(labels_per_protein.median(), color='orange', linestyle='--',
                      linewidth=2, label=f'Median: {labels_per_protein.median():.1f}')
    axes[0, 0].set_xlabel('Number of Labels', fontsize=12)
    axes[0, 0].set_ylabel('Number of Proteins', fontsize=12)
    axes[0, 0].set_title('Labels per Protein Distribution', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Cumulative distribution
    sorted_labels = np.sort(labels_per_protein)
    cumulative = np.arange(1, len(sorted_labels) + 1) / len(sorted_labels) * 100
    axes[0, 1].plot(sorted_labels, cumulative, color='steelblue', linewidth=2)
    axes[0, 1].set_xlabel('Number of Labels', fontsize=12)
    axes[0, 1].set_ylabel('Cumulative Percentage (%)', fontsize=12)
    axes[0, 1].set_title('Cumulative Distribution', fontsize=14, fontweight='bold')
    axes[0, 1].grid(alpha=0.3)
    axes[0, 1].axhline(50, color='red', linestyle='--', alpha=0.5, label='50%')
    axes[0, 1].axhline(90, color='orange', linestyle='--', alpha=0.5, label='90%')
    axes[0, 1].legend()
    
    # Bar chart by ranges
    ranges = [(1, 5), (6, 10), (11, 20), (21, 30), (31, 50), 
              (51, 75), (76, 100), (101, max_labels)]
    range_counts = []
    range_labels = []
    
    for start, end in ranges:
        count = ((labels_per_protein >= start) & (labels_per_protein <= end)).sum()
        range_counts.append(count)
        if end == max_labels:
            range_labels.append(f'{start}+')
        else:
            range_labels.append(f'{start}-{end}')
    
    colors_gradient = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(range_counts)))
    axes[1, 0].bar(range_labels, range_counts, color=colors_gradient, 
                  alpha=0.8, edgecolor='black')
    axes[1, 0].set_xlabel('Label Range', fontsize=12)
    axes[1, 0].set_ylabel('Number of Proteins', fontsize=12)
    axes[1, 0].set_title('Proteins by Label Range', fontsize=14, fontweight='bold')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    # Statistics table
    axes[1, 1].axis('off')
    stats_data = [
        ['Metric', 'Value'],
        ['Total Proteins', f'{len(labels_per_protein):,}'],
        ['Mean Labels', f'{labels_per_protein.mean():.2f}'],
        ['Median Labels', f'{labels_per_protein.median():.0f}'],
        ['Std Dev', f'{labels_per_protein.std():.2f}'],
        ['Min Labels', f'{labels_per_protein.min()}'],
        ['Max Labels', f'{labels_per_protein.max()}'],
        ['25th Percentile', f'{labels_per_protein.quantile(0.25):.0f}'],
        ['75th Percentile', f'{labels_per_protein.quantile(0.75):.0f}'],
        ['Proteins with >50 labels', f'{(labels_per_protein > 50).sum():,}'],
    ]
    
    table = axes[1, 1].table(cellText=stats_data, loc='center', cellLoc='left',
                            colWidths=[0.6, 0.4])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2.5)
    
    for i in range(2):
        table[(0, i)].set_facecolor('#FF6B6B')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    axes[1, 1].set_title('Statistics Summary', fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print detailed statistics
    print(f"\n{'='*60}")
    print(f"LABELS PER PROTEIN STATISTICS")
    print(f"{'='*60}")
    print(f"Total proteins: {len(labels_per_protein):,}")
    print(f"Mean labels: {labels_per_protein.mean():.2f}")
    print(f"Median labels: {labels_per_protein.median():.0f}")
    print(f"Std deviation: {labels_per_protein.std():.2f}")
    print(f"Min labels: {labels_per_protein.min()}")
    print(f"Max labels: {labels_per_protein.max()}")
    print(f"\nPercentiles:")
    for p in [10, 25, 50, 75, 90, 95, 99]:
        print(f"  {p}th: {labels_per_protein.quantile(p/100):.0f}")
    print(f"{'='*60}\n")


In [ ]:
plot_labels_per_protein(annotations, save_path="/kaggle/working/labels_per_protein.png")

In [ ]:
def plot_ontology_distribution(annotations: pd.DataFrame,
                               save_path: str = None):
    """
    Plot distribution of annotations across ontologies.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology columns
        save_path: Optional path to save figure
    """
    # Create a clean copy of the dataframe, skipping the header row
    if annotations.iloc[0, 0] == 'EntryID' and annotations.iloc[0, 1] == 'term' and annotations.iloc[0, 2] == 'aspect':
        clean_annotations = annotations.iloc[1:].copy()
        clean_annotations.columns = ['protein_id', 'go_term', 'ontology']
    else:
        clean_annotations = annotations.copy()
    
    # Map aspect codes to ontology codes
    aspect_to_ontology = {
        'P': 'BPO',  # Biological Process
        'F': 'MFO',  # Molecular Function  
        'C': 'CCO'   # Cellular Component
    }
    
    # Convert aspect codes to ontology codes
    clean_annotations['ontology'] = clean_annotations['ontology'].map(aspect_to_ontology)
    
    # Check if annotations DataFrame is empty
    if clean_annotations.empty:
        print("Warning: annotations DataFrame is empty. No data to plot.")
        return
    
    # Check for required columns
    required_columns = ['protein_id', 'go_term', 'ontology']
    missing_columns = [col for col in required_columns if col not in clean_annotations.columns]
    if missing_columns:
        print(f"Warning: Missing required columns: {missing_columns}")
        return
    
    # Filter out any rows with missing values in required columns
    clean_annotations = clean_annotations.dropna(subset=required_columns)
    
    # Check if we have any data left after filtering
    if clean_annotations.empty:
        print("Warning: No valid data after filtering missing values.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Count by ontology
    ontology_counts = clean_annotations['ontology'].value_counts()
    ontology_order = ['BPO', 'MFO', 'CCO']
    
    # Reindex with the desired order, but only include existing ontologies
    existing_ontologies = [ont for ont in ontology_order if ont in ontology_counts.index]
    ontology_counts = ontology_counts.reindex(ontology_order, fill_value=0)
    
    ontology_names = {
        'BPO': 'Biological Process',
        'MFO': 'Molecular Function',
        'CCO': 'Cellular Component'
    }
    
    colors = {'BPO': '#EF4444', 'MFO': '#3B82F6', 'CCO': '#10B981'}
    
    # Bar chart - annotations by ontology
    axes[0, 0].bar(ontology_counts.index, ontology_counts.values,
                  color=[colors[ont] for ont in ontology_counts.index],
                  alpha=0.8, edgecolor='black', linewidth=2)
    axes[0, 0].set_xlabel('Ontology', fontsize=12)
    axes[0, 0].set_ylabel('Number of Annotations', fontsize=12)
    axes[0, 0].set_title('Annotations by Ontology', fontsize=14, fontweight='bold')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Add count labels on bars
    for i, (ont, count) in enumerate(ontology_counts.items()):
        axes[0, 0].text(i, count, f'{count:,}\n({count/ontology_counts.sum()*100:.1f}%)',
                       ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Pie chart
    axes[0, 1].pie(ontology_counts.values, 
                  labels=[f"{ont}\n{ontology_names[ont]}" for ont in ontology_counts.index],
                  autopct='%1.1f%%', colors=[colors[ont] for ont in ontology_counts.index],
                  startangle=90, textprops={'fontsize': 11, 'weight': 'bold'})
    axes[0, 1].set_title('Ontology Distribution', fontsize=14, fontweight='bold')
    
    # Unique terms per ontology
    unique_terms = clean_annotations.groupby('ontology')['go_term'].nunique()
    unique_terms = unique_terms.reindex(ontology_order, fill_value=0)
    
    axes[1, 0].bar(unique_terms.index, unique_terms.values,
                  color=[colors[ont] for ont in unique_terms.index],
                  alpha=0.8, edgecolor='black', linewidth=2)
    axes[1, 0].set_xlabel('Ontology', fontsize=12)
    axes[1, 0].set_ylabel('Number of Unique GO Terms', fontsize=12)
    axes[1, 0].set_title('Unique Terms by Ontology', fontsize=14, fontweight='bold')
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    # Add count labels
    for i, (ont, count) in enumerate(unique_terms.items()):
        axes[1, 0].text(i, count, f'{count:,}', ha='center', va='bottom',
                       fontsize=10, fontweight='bold')
    
    # Unique proteins per ontology
    unique_proteins = clean_annotations.groupby('ontology')['protein_id'].nunique()
    unique_proteins = unique_proteins.reindex(ontology_order, fill_value=0)
    
    axes[1, 1].bar(unique_proteins.index, unique_proteins.values,
                  color=[colors[ont] for ont in unique_proteins.index],
                  alpha=0.8, edgecolor='black', linewidth=2)
    axes[1, 1].set_xlabel('Ontology', fontsize=12)
    axes[1, 1].set_ylabel('Number of Unique Proteins', fontsize=12)
    axes[1, 1].set_title('Unique Proteins by Ontology', fontsize=14, fontweight='bold')
    axes[1, 1].grid(axis='y', alpha=0.3)
    
    # Add count labels
    for i, (ont, count) in enumerate(unique_proteins.items()):
        axes[1, 1].text(i, count, f'{count:,}', ha='center', va='bottom',
                       fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print statistics
    print(f"\n{'='*60}")
    print(f"ONTOLOGY DISTRIBUTION")
    print(f"{'='*60}")
    total = ontology_counts.sum()
    for ont in ontology_order:
        print(f"\n{ont} - {ontology_names[ont]}:")
        print(f"  Annotations: {ontology_counts[ont]:,} ({ontology_counts[ont]/total*100:.1f}%)")
        print(f"  Unique terms: {unique_terms[ont]:,}")
        print(f"  Unique proteins: {unique_proteins[ont]:,}")
    print(f"\nTotal annotations: {total:,}")
    print(f"{'='*60}\n")
    
    return clean_annotations

In [ ]:
plot_ontology_distribution(annotations, save_path="/kaggle/working/ontology_distribution.png")

In [ ]:
def plot_term_frequency(annotations: pd.DataFrame,
                       top_n: int = 20,
                       min_freq: int = 1,
                       save_path: str = None):
    """
    Plot GO term frequency distribution.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology columns
        top_n: Number of top terms to display
        min_freq: Minimum frequency to include
        save_path: Optional path to save figure
    """
    term_freq = annotations['go_term'].value_counts()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Top N terms (horizontal bar chart)
    top_terms = term_freq.head(top_n)
    y_pos = np.arange(len(top_terms))
    
    axes[0, 0].barh(y_pos, top_terms.values, color='steelblue', 
                   alpha=0.8, edgecolor='black')
    axes[0, 0].set_yticks(y_pos)
    axes[0, 0].set_yticklabels(top_terms.index, fontsize=9)
    axes[0, 0].set_xlabel('Number of Proteins', fontsize=12)
    axes[0, 0].set_title(f'Top {top_n} Most Frequent GO Terms', 
                        fontsize=14, fontweight='bold')
    axes[0, 0].invert_yaxis()
    axes[0, 0].grid(axis='x', alpha=0.3)
    
    # Add count labels
    for i, v in enumerate(top_terms.values):
        axes[0, 0].text(v, i, f' {v:,}', va='center', fontsize=9)
    
    # Frequency distribution (log scale)
    axes[0, 1].hist(np.log10(term_freq.values), bins=50, 
                   color='coral', alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('log10(Term Frequency)', fontsize=12)
    axes[0, 1].set_ylabel('Number of Terms', fontsize=12)
    axes[0, 1].set_title('Term Frequency Distribution (Log Scale)', 
                        fontsize=14, fontweight='bold')
    axes[0, 1].grid(alpha=0.3)
    
    # Cumulative frequency
    sorted_freq = np.sort(term_freq.values)[::-1]
    cumulative = np.cumsum(sorted_freq) / sorted_freq.sum() * 100
    
    axes[1, 0].plot(range(len(cumulative)), cumulative, 
                   color='green', linewidth=2)
    axes[1, 0].set_xlabel('Number of Terms (Ranked)', fontsize=12)
    axes[1, 0].set_ylabel('Cumulative Coverage (%)', fontsize=12)
    axes[1, 0].set_title('Cumulative Term Coverage', fontsize=14, fontweight='bold')
    axes[1, 0].axhline(80, color='red', linestyle='--', 
                      alpha=0.5, label='80% coverage')
    axes[1, 0].axhline(90, color='orange', linestyle='--', 
                      alpha=0.5, label='90% coverage')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Class imbalance statistics
    axes[1, 1].axis('off')
    
    imbalance_ratio = term_freq.iloc[0] / term_freq.iloc[-1]
    rare_terms = (term_freq < 10).sum()
    singleton_terms = (term_freq == 1).sum()
    coverage_80 = np.where(cumulative >= 80)[0][0] if np.any(cumulative >= 80) else len(cumulative)
    
    stats_data = [
        ['Metric', 'Value'],
        ['Total Unique Terms', f'{len(term_freq):,}'],
        ['Most Common Term', f'{term_freq.iloc[0]:,}'],
        ['Least Common Term', f'{term_freq.iloc[-1]}'],
        ['Imbalance Ratio', f'{imbalance_ratio:.1f}x'],
        ['Terms with <10 proteins', f'{rare_terms:,} ({rare_terms/len(term_freq)*100:.1f}%)'],
        ['Singleton Terms', f'{singleton_terms:,} ({singleton_terms/len(term_freq)*100:.1f}%)'],
        ['Terms for 80% coverage', f'{coverage_80:,}'],
        ['Mean Frequency', f'{term_freq.mean():.1f}'],
        ['Median Frequency', f'{term_freq.median():.0f}'],
    ]
    
    table = axes[1, 1].table(cellText=stats_data, loc='center', cellLoc='left',
                            colWidths=[0.65, 0.35])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2.5)
    
    for i in range(2):
        table[(0, i)].set_facecolor('#FF6B6B')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    axes[1, 1].set_title('Class Imbalance Statistics', 
                        fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print detailed statistics
    print(f"\n{'='*60}")
    print(f"GO TERM FREQUENCY ANALYSIS")
    print(f"{'='*60}")
    print(f"Total unique terms: {len(term_freq):,}")
    print(f"Total annotations: {term_freq.sum():,}")
    print(f"\nFrequency Statistics:")
    print(f"  Most common: {term_freq.iloc[0]:,} ({term_freq.index[0]})")
    print(f"  Least common: {term_freq.iloc[-1]} ({term_freq.index[-1]})")
    print(f"  Imbalance ratio: {imbalance_ratio:.1f}x")
    print(f"  Mean: {term_freq.mean():.1f}")
    print(f"  Median: {term_freq.median():.0f}")
    print(f"\nClass Imbalance:")
    print(f"  Terms with <10 proteins: {rare_terms:,} ({rare_terms/len(term_freq)*100:.1f}%)")
    print(f"  Singleton terms: {singleton_terms:,} ({singleton_terms/len(term_freq)*100:.1f}%)")
    print(f"  Terms for 80% coverage: {coverage_80:,}")
    print(f"{'='*60}\n")


In [ ]:
plot_term_frequency(annotations, save_path="/kaggle/working/term_frequency.png")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from collections import Counter
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')


def analyze_label_cooccurrence(annotations: pd.DataFrame, 
                               min_support: int = None,
                               auto_threshold: bool = True) -> pd.DataFrame:
    """
    Analyze which GO terms frequently co-occur with adaptive thresholding.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology columns
        min_support: Minimum number of proteins for pair to be included
        auto_threshold: If True, automatically determine min_support
        
    Returns:
        DataFrame with term pairs and co-occurrence counts
    """
    if len(annotations) == 0:
        print("Warning: Empty annotations DataFrame")
        return pd.DataFrame(columns=['term1', 'term2', 'count', 'ont1', 'ont2'])
    
    # Group terms by protein
    protein_terms = annotations.groupby('protein_id')['go_term'].apply(list).to_dict()
    
    # Create ontology mapping for later use
    term_to_ont = annotations.drop_duplicates('go_term').set_index('go_term')['ontology'].to_dict()
    
    # Count pairs
    pair_counts = Counter()
    for terms in protein_terms.values():
        if len(terms) > 1:
            for i, term1 in enumerate(terms):
                for term2 in terms[i+1:]:
                    pair = tuple(sorted([term1, term2]))
                    pair_counts[pair] += 1
    
    if len(pair_counts) == 0:
        print("Warning: No co-occurring term pairs found")
        return pd.DataFrame(columns=['term1', 'term2', 'count', 'ont1', 'ont2'])
    
    # Auto-determine min_support if requested
    if auto_threshold and min_support is None:
        counts = list(pair_counts.values())
        min_support = max(1, int(np.percentile(counts, 50)))  # Use median as threshold
        print(f"Auto-determined min_support: {min_support} (median of co-occurrence counts)")
   
    elif min_support is None:
        min_support = 10
    
    # Filter and create DataFrame
    cooccurrence = pd.DataFrame([
        {
            'term1': pair[0], 
            'term2': pair[1], 
            'count': count,
            'ont1': term_to_ont.get(pair[0], 'Unknown'),
            'ont2': term_to_ont.get(pair[1], 'Unknown')
        }
        for pair, count in pair_counts.items()
        if count >= min_support
    ])
    
    if len(cooccurrence) == 0:
        # Try with lower threshold
        new_min = max(1, min_support // 2)
        print(f"No pairs found with min_support={min_support}. Retrying with {new_min}...")
        cooccurrence = pd.DataFrame([
            {
                'term1': pair[0], 
                'term2': pair[1], 
                'count': count,
                'ont1': term_to_ont.get(pair[0], 'Unknown'),
                'ont2': term_to_ont.get(pair[1], 'Unknown')
            }
            for pair, count in pair_counts.items()
            if count >= new_min
        ])
        min_support = new_min
    
    if len(cooccurrence) > 0:
        cooccurrence = cooccurrence.sort_values('count', ascending=False)
        
        # Add pair type column
        cooccurrence['pair_type'] = cooccurrence.apply(
            lambda row: 'within_ontology' if row['ont1'] == row['ont2'] else 'cross_ontology',
            axis=1
        )
        
        print(f"\n{'='*60}")
        print(f"LABEL CO-OCCURRENCE ANALYSIS")
        print(f"{'='*60}")
        print(f"Total unique term pairs: {len(pair_counts):,}")
        print(f"Pairs with ≥{min_support} co-occurrences: {len(cooccurrence):,}")
        print(f"  Within-ontology pairs: {(cooccurrence['pair_type'] == 'within_ontology').sum():,}")
        print(f"  Cross-ontology pairs: {(cooccurrence['pair_type'] == 'cross_ontology').sum():,}")
        
        # Statistics by ontology pair type
        print(f"\nPairs by Ontology Combination:")
        for ont1 in ['BPO', 'MFO', 'CCO']:
            for ont2 in ['BPO', 'MFO', 'CCO']:
                if ont1 <= ont2:  # Avoid duplicates
                    mask = ((cooccurrence['ont1'] == ont1) & (cooccurrence['ont2'] == ont2)) | \
                           ((cooccurrence['ont1'] == ont2) & (cooccurrence['ont2'] == ont1))
                    count = mask.sum()
                    if count > 0:
                        label = f"{ont1}-{ont2}" if ont1 != ont2 else ont1
                        print(f"  {label}: {count:,} pairs")
        
        print(f"\nTop 10 Co-occurring Term Pairs:")
        for idx, row in cooccurrence.head(10).iterrows():
            print(f"  {row['term1']} ({row['ont1']}) ↔ {row['term2']} ({row['ont2']}): {row['count']:,}")
        print(f"{'='*60}\n")
    else:
        print(f"\nNo term pairs found with ≥{min_support} co-occurrences")
    
    return cooccurrence



In [ ]:
import seaborn as sns
import networkx as nx
from matplotlib.patches import FancyBboxPatch
import numpy as np

def visualize_cooccurrence(cooccurrence_df: pd.DataFrame, 
                          annotations: pd.DataFrame,
                          top_n: int = 20,
                          save_path: str = None):
    """
    Visualize GO term co-occurrence analysis results.
    
    Args:
        cooccurrence_df: DataFrame from analyze_label_cooccurrence
        annotations: Original annotations DataFrame for context
        top_n: Number of top pairs to visualize
        save_path: Optional path to save figure
    """
    if cooccurrence_df.empty:
        print("No co-occurrence data to visualize.")
        return
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(20, 15))
    
    # Define grid layout
    gs = fig.add_gridspec(3, 2, height_ratios=[1, 1, 0.8])
    
    # 1. Heatmap of top co-occurring pairs
    ax1 = fig.add_subplot(gs[0, 0])
    plot_cooccurrence_heatmap(cooccurrence_df, top_n, ax1)
    
    # 2. Network graph
    ax2 = fig.add_subplot(gs[0, 1])
    plot_cooccurrence_network(cooccurrence_df, top_n, ax2)
    
    # 3. Distribution of co-occurrence counts
    ax3 = fig.add_subplot(gs[1, 0])
    plot_cooccurrence_distribution(cooccurrence_df, ax3)
    
    # 4. Top terms by overall frequency
    ax4 = fig.add_subplot(gs[1, 1])
    plot_term_frequency(annotations, cooccurrence_df, top_n, ax4)
    
    # 5. Statistics table
    ax5 = fig.add_subplot(gs[2, :])
    plot_cooccurrence_stats(cooccurrence_df, annotations, ax5)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

def plot_cooccurrence_heatmap(cooccurrence_df: pd.DataFrame, top_n: int, ax):
    """Plot heatmap of top co-occurring term pairs."""
    top_pairs = cooccurrence_df.head(top_n).copy()
    
    # Get unique terms from top pairs
    all_terms = set(top_pairs['term1']).union(set(top_pairs['term2']))
    term_list = sorted(list(all_terms))
    
    # Create co-occurrence matrix
    cooccurrence_matrix = pd.DataFrame(0, index=term_list, columns=term_list)
    
    for _, row in top_pairs.iterrows():
        cooccurrence_matrix.loc[row['term1'], row['term2']] = row['count']
        cooccurrence_matrix.loc[row['term2'], row['term1']] = row['count']
    
    # Plot heatmap
    mask = np.triu(np.ones_like(cooccurrence_matrix, dtype=bool), k=1)
    sns.heatmap(cooccurrence_matrix, mask=mask, cmap='YlOrRd', 
                annot=True, fmt='d', ax=ax, cbar_kws={'label': 'Co-occurrence Count'})
    
    ax.set_title(f'Top {top_n} GO Term Co-occurrence Heatmap', fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    ax.tick_params(axis='y', rotation=0)

def plot_cooccurrence_network(cooccurrence_df: pd.DataFrame, top_n: int, ax):
    """Plot network graph of co-occurring terms."""
    top_pairs = cooccurrence_df.head(top_n).copy()
    
    # Create graph
    G = nx.Graph()
    
    # Add edges with weights
    for _, row in top_pairs.iterrows():
        G.add_edge(row['term1'], row['term2'], weight=row['count'])
    
    if len(G.edges()) == 0:
        ax.text(0.5, 0.5, 'No co-occurrence pairs to display', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Co-occurrence Network', fontsize=14, fontweight='bold')
        return
    
    # Calculate node positions
    pos = nx.spring_layout(G, k=1, iterations=50)
    
    # Calculate node sizes based on degree
    degrees = dict(G.degree())
    node_sizes = [300 + degrees[node] * 100 for node in G.nodes()]
    
    # Calculate edge widths based on co-occurrence count
    edge_weights = [G[u][v]['weight'] / 10 for u, v in G.edges()]
    
    # Draw network
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, 
                          node_color='lightblue', alpha=0.9, ax=ax)
    nx.draw_networkx_edges(G, pos, width=edge_weights, 
                          alpha=0.6, edge_color='gray', ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=8, ax=ax)
    
    # Add edge labels for top 5 strongest connections
    edge_labels = {(u, v): f"{G[u][v]['weight']}" 
                   for u, v in list(G.edges())[:5]}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, 
                                font_size=7, ax=ax)
    
    ax.set_title(f'Co-occurrence Network (Top {top_n} Pairs)', 
                fontsize=14, fontweight='bold')
    ax.axis('off')

def plot_cooccurrence_distribution(cooccurrence_df: pd.DataFrame, ax):
    """Plot distribution of co-occurrence counts."""
    counts = cooccurrence_df['count']
    
    ax.hist(counts, bins=30, alpha=0.7, color='skyblue', 
            edgecolor='black', linewidth=0.5)
    ax.axvline(counts.mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {counts.mean():.1f}')
    ax.axvline(counts.median(), color='orange', linestyle='--', 
               linewidth=2, label=f'Median: {counts.median():.1f}')
    
    ax.set_xlabel('Co-occurrence Count', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title('Distribution of Co-occurrence Counts', 
                fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

def plot_term_frequency(annotations: pd.DataFrame, cooccurrence_df: pd.DataFrame, top_n: int, ax):
    """Plot most frequent terms in co-occurring pairs."""
    # Get all unique terms from co-occurrence pairs
    all_cooccur_terms = set(cooccurrence_df['term1']).union(set(cooccurrence_df['term2']))
    
    # Calculate overall frequency of these terms
    term_freq = annotations[annotations['go_term'].isin(all_cooccur_terms)]['go_term'].value_counts()
    top_terms = term_freq.head(top_n)
    
    # Get colors based on ontology
    ontology_map = annotations.set_index('go_term')['ontology'].to_dict()
    colors = []
    for term in top_terms.index:
        ont = ontology_map.get(term, 'Unknown')
        if ont == 'BPO': colors.append('#EF4444')
        elif ont == 'MFO': colors.append('#3B82F6')
        elif ont == 'CCO': colors.append('#10B981')
        else: colors.append('gray')
    
    bars = ax.barh(range(len(top_terms)), top_terms.values, color=colors, alpha=0.8)
    ax.set_yticks(range(len(top_terms)))
    ax.set_yticklabels([f"{term}" for term in top_terms.index], fontsize=9)
    ax.invert_yaxis()
    ax.set_xlabel('Frequency in Dataset', fontsize=12)
    ax.set_title(f'Top {top_n} Terms in Co-occurrence Pairs', 
                fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, count) in enumerate(zip(bars, top_terms.values)):
        ax.text(bar.get_width() + bar.get_width() * 0.01, bar.get_y() + bar.get_height()/2,
                f'{count:,}', ha='left', va='center', fontsize=8)

def plot_cooccurrence_stats(cooccurrence_df: pd.DataFrame, annotations: pd.DataFrame, ax):
    """Plot statistics table."""
    ax.axis('off')
    
    # Calculate statistics
    total_pairs = len(cooccurrence_df)
    total_cooccurrences = cooccurrence_df['count'].sum()
    avg_cooccurrence = cooccurrence_df['count'].mean()
    max_cooccurrence = cooccurrence_df['count'].max()
    
    # Get unique terms involved in co-occurrence
    cooccur_terms = set(cooccurrence_df['term1']).union(set(cooccurrence_df['term2']))
    
    # Calculate statistics by ontology
    ontologies = ['BPO', 'MFO', 'CCO']
    stats_by_ontology = []
    
    for ont in ontologies:
        ont_terms = set(annotations[annotations['ontology'] == ont]['go_term'])
        ont_cooccur_terms = cooccur_terms.intersection(ont_terms)
        ont_pairs = cooccurrence_df[
            (cooccurrence_df['term1'].isin(ont_terms)) | 
            (cooccurrence_df['term2'].isin(ont_terms))
        ]
        stats_by_ontology.append({
            'ontology': ont,
            'terms_in_cooccurrence': len(ont_cooccur_terms),
            'pairs_involving_ontology': len(ont_pairs)
        })
    
    # Create table data
    table_data = [
        ['Total Pairs', f'{total_pairs:,}'],
        ['Total Co-occurrences', f'{total_cooccurrences:,}'],
        ['Average Co-occurrence', f'{avg_cooccurrence:.1f}'],
        ['Maximum Co-occurrence', f'{max_cooccurrence:,}'],
        ['Unique Terms in Pairs', f'{len(cooccur_terms):,}'],
        ['', ''],
        ['Ontology', 'Terms in Pairs / Pairs Involving'],
    ]
    
    for stats in stats_by_ontology:
        table_data.append([
            stats['ontology'],
            f"{stats['terms_in_cooccurrence']:,} / {stats['pairs_involving_ontology']:,}"
        ])
    
    # Create table
    table = ax.table(cellText=table_data, 
                    cellLoc='left',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style header row
    for i in range(len(table_data)):
        for j in range(len(table_data[0])):
            if i == 0 or i == 6:  # Header rows
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
            elif i > 6:  # Ontology rows
                if j == 0:
                    table[(i, j)].set_facecolor('#E5E7EB')
    
    ax.set_title('Co-occurrence Analysis Statistics', 
                fontsize=16, fontweight='bold', pad=20)

# Updated wrapper function that combines analysis and visualization
def analyze_and_visualize_cooccurrence(annotations: pd.DataFrame, 
                                      min_support: int = 10,
                                      auto_threshold: bool = True,
                                      top_n: int = 20,
                                      save_path: str = None) -> pd.DataFrame:
    """
    Analyze GO term co-occurrence and create visualizations.
    
    Args:
        annotations: DataFrame with annotations
        min_support: Minimum number of proteins for pair to be included
        top_n: Number of top pairs to visualize
        save_path: Optional path to save figure
        
    Returns:
        DataFrame with term pairs and co-occurrence counts
    """
    # Perform analysis
    cooccurrence_df = analyze_label_cooccurrence(annotations, min_support, auto_threshold)
    
    # Create visualizations if we have data
    if not cooccurrence_df.empty:
        visualize_cooccurrence(cooccurrence_df, annotations, top_n, save_path)
    
    return cooccurrence_df

In [ ]:
# Pre-process your annotations DataFrame first
def prepare_annotations_for_analysis(annotations):
    """Prepare annotations DataFrame for co-occurrence analysis."""
    clean_annotations = annotations.copy()
    
    # Skip header row if present
    if clean_annotations.iloc[0, 0] == 'EntryID':
        clean_annotations = clean_annotations.iloc[1:].copy()
        clean_annotations.columns = ['protein_id', 'go_term', 'ontology']
    
    # Map aspect codes to ontology codes
    aspect_to_ontology = {
        'P': 'BPO',  # Biological Process
        'F': 'MFO',  # Molecular Function  
        'C': 'CCO'   # Cellular Component
    }
    
    clean_annotations['ontology'] = clean_annotations['ontology'].map(aspect_to_ontology)
    return clean_annotations


In [ ]:
# Usage:
annotations = prepare_annotations_for_analysis(annotations)

In [ ]:
analyze_and_visualize_cooccurrence(
        annotations, 
        auto_threshold=True,
        top_n=20,
        save_path='/kaggle/working/co-occurence.png')

# Graph Analysis Functions (For Ontologies)

In [ ]:
import networkx as nx
import numpy as np
from typing import Dict
import matplotlib.pyplot as plt

def analyze_go_hierarchy(graph: nx.DiGraph, annotations: pd.DataFrame) -> Dict:
    """
    Analyze GO ontology graph structure.
    
    Args:
        graph: NetworkX graph instance of GO ontology
        annotations: DataFrame with annotations (to find annotated terms)
        
    Returns:
        Dictionary with graph statistics
    """
    stats = {}
    
    # Basic graph statistics
    stats['num_nodes'] = graph.number_of_nodes()
    stats['num_edges'] = graph.number_of_edges()
    
    # Find annotated terms
    annotated_terms = set(annotations['go_term'].unique())
    annotated_in_graph = annotated_terms & set(graph.nodes())
    
    stats['annotated_terms_in_graph'] = len(annotated_in_graph)
    stats['annotated_terms_not_in_graph'] = len(annotated_terms - annotated_in_graph)
    
    # Calculate term depths (distance from root) - FIXED VERSION
    root_terms = {
        'GO:0008150',  # biological_process
        'GO:0003674',  # molecular_function
        'GO:0005575',  # cellular_component
    }
    
    # Find actual roots (nodes with no incoming edges)
    actual_roots = [node for node in graph.nodes() if graph.in_degree(node) == 0]
    stats['actual_roots'] = actual_roots
    
    depths = {}
    valid_roots_found = 0
    
    for root in root_terms:
        if root in graph:
            try:
                # Use reverse graph for depth calculation (distance TO root)
                reverse_graph = graph.reverse()
                depth_dict = nx.single_source_shortest_path_length(reverse_graph, root)
                if depth_dict:  # Only include if we found reachable nodes
                    depths[root] = depth_dict
                    valid_roots_found += 1
                    print(f"Found root {root} with {len(depth_dict)} reachable terms")
            except nx.NetworkXError as e:
                print(f"Warning: Root term {root} error: {e}")
                continue
    
    # Alternative: Calculate depth from any root
    if not depths:
        print("Trying alternative depth calculation from any root...")
        # Use nodes with in_degree 0 as roots
        for root in actual_roots[:3]:  # Use first 3 actual roots
            try:
                reverse_graph = graph.reverse()
                depth_dict = nx.single_source_shortest_path_length(reverse_graph, root)
                if len(depth_dict) > 10:  # Only include meaningful roots
                    depths[root] = depth_dict
                    print(f"Using root {root} with {len(depth_dict)} reachable terms")
                    break
            except:
                continue
    
    if depths:
        all_depths = []
        for depth_dict in depths.values():
            all_depths.extend(depth_dict.values())
        
        if all_depths:
            stats['term_depth'] = {
                'mean': np.mean(all_depths),
                'max': max(all_depths),
                'min': min(all_depths),
                'std': np.std(all_depths),
                'num_measured_terms': len(all_depths)
            }
            print(f"Depth calculation: measured {len(all_depths)} terms")
        else:
            stats['term_depth'] = {'error': 'No depth values calculated'}
    else:
        stats['term_depth'] = {'error': 'No valid roots found for depth calculation'}
    
    # Additional graph analysis
    if graph.number_of_nodes() > 0:
        # Check if graph is a DAG (should be for ontology)
        stats['is_dag'] = nx.is_directed_acyclic_graph(graph)
        
        # Calculate connectivity on a sample for large graphs
        if graph.number_of_nodes() > 10000:
            print("Large graph detected, using sampling for connectivity analysis...")
            # Sample nodes for connectivity analysis
            sample_nodes = list(graph.nodes())[:1000]
            subgraph = graph.subgraph(sample_nodes)
            stats['is_weakly_connected'] = nx.is_weakly_connected(subgraph)
            stats['is_strongly_connected'] = nx.is_strongly_connected(subgraph)
            stats['connectivity_note'] = 'Based on 1000 node sample'
        else:
            stats['is_weakly_connected'] = nx.is_weakly_connected(graph)
            stats['is_strongly_connected'] = nx.is_strongly_connected(graph)
        
        # Node degree analysis (optimized for large graphs)
        if graph.number_of_nodes() > 50000:
            print("Large graph detected, sampling for degree analysis...")
            sample_size = min(10000, graph.number_of_nodes())
            sample_nodes = np.random.choice(list(graph.nodes()), sample_size, replace=False)
            in_degrees = [graph.in_degree(node) for node in sample_nodes]
            out_degrees = [graph.out_degree(node) for node in sample_nodes]
            stats['degree_sample_size'] = sample_size
        else:
            in_degrees = [d for n, d in graph.in_degree()]
            out_degrees = [d for n, d in graph.out_degree()]
        
        stats['degree_analysis'] = {
            'in_degree_mean': np.mean(in_degrees) if in_degrees else 0,
            'in_degree_max': max(in_degrees) if in_degrees else 0,
            'out_degree_mean': np.mean(out_degrees) if out_degrees else 0,
            'out_degree_max': max(out_degrees) if out_degrees else 0,
            'in_degree_median': np.median(in_degrees) if in_degrees else 0,
            'out_degree_median': np.median(out_degrees) if out_degrees else 0,
        }
        
        # Component analysis
        if graph.number_of_nodes() <= 50000:
            weakly_components = list(nx.weakly_connected_components(graph))
            stats['weakly_connected_components'] = len(weakly_components)
            if weakly_components:
                stats['largest_component_size'] = len(max(weakly_components, key=len))
        else:
            stats['component_note'] = 'Skipped for large graph'
    
    print("\nGO Ontology Graph Analysis:")
    print(f"Total terms in ontology: {stats['num_nodes']:,}")
    print(f"Total relationships: {stats['num_edges']:,}")
    print(f"Annotated terms in training: {stats['annotated_terms_in_graph']:,}")
    print(f"Annotated terms not in graph: {stats['annotated_terms_not_in_graph']:,}")
    
    if 'term_depth' in stats and 'error' not in stats['term_depth']:
        depth_stats = stats['term_depth']
        print(f"\nTerm depth statistics:")
        print(f"  Mean: {depth_stats['mean']:.2f}")
        print(f"  Max: {depth_stats['max']}")
        print(f"  Min: {depth_stats['min']}")
        print(f"  Std: {depth_stats['std']:.2f}")
        print(f"  Terms measured: {depth_stats['num_measured_terms']:,}")
    elif 'term_depth' in stats:
        print(f"\nDepth analysis: {stats['term_depth']['error']}")
    
    if 'degree_analysis' in stats:
        deg = stats['degree_analysis']
        print(f"\nDegree analysis:")
        print(f"  Mean in-degree: {deg['in_degree_mean']:.2f}")
        print(f"  Median in-degree: {deg['in_degree_median']:.1f}")
        print(f"  Max in-degree: {deg['in_degree_max']}")
        print(f"  Mean out-degree: {deg['out_degree_mean']:.2f}")
        print(f"  Median out-degree: {deg['out_degree_median']:.1f}")
        print(f"  Max out-degree: {deg['out_degree_max']}")
        if 'degree_sample_size' in stats:
            print(f"  (Based on {stats['degree_sample_size']:,} node sample)")
    
    print(f"\nGraph properties:")
    print(f"  Is DAG: {stats.get('is_dag', 'N/A')}")
    print(f"  Weakly connected: {stats.get('is_weakly_connected', 'N/A')}")
    print(f"  Strongly connected: {stats.get('is_strongly_connected', 'N/A')}")
    if 'weakly_connected_components' in stats:
        print(f"  Weakly connected components: {stats['weakly_connected_components']}")
        print(f"  Largest component size: {stats.get('largest_component_size', 'N/A'):,}")
    
    return stats

# OPTIMIZED Visualization function for GO hierarchy
def visualize_go_hierarchy(graph: nx.DiGraph, annotations: pd.DataFrame, 
                          save_path: str = None, max_plot_terms: int = 1000):
    """
    Visualize GO ontology graph structure - OPTIMIZED VERSION.
    
    Args:
        graph: NetworkX graph instance of GO ontology
        annotations: DataFrame with annotations
        save_path: Optional path to save figure
        max_plot_terms: Maximum number of terms to include in plots
    """
    print("Generating optimized visualizations...")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Optimized depth distribution
    ax1 = axes[0, 0]
    plot_depth_distribution_optimized(graph, ax1)
    
    # 2. Optimized degree distribution
    ax2 = axes[0, 1]
    plot_degree_distribution_optimized(graph, ax2)
    
    # 3. Statistics summary
    ax3 = axes[1, 0]
    plot_hierarchy_stats_optimized(graph, annotations, ax3)
    
    # 4. Annotated terms overview (fast alternative to network plot)
    ax4 = axes[1, 1]
    plot_annotated_overview(graph, annotations, ax4)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved visualization to {save_path}")
    
    plt.show()

def plot_depth_distribution_optimized(graph: nx.DiGraph, ax):
    """Optimized depth distribution plot."""
    print("Calculating depth distribution...")
    
    # Use reverse graph for depth calculation
    reverse_graph = graph.reverse()
    
    # Try standard roots first
    root_terms = ['GO:0008150', 'GO:0003674', 'GO:0005575']
    depths = []
    
    for root in root_terms:
        if root in reverse_graph:
            try:
                depth_dict = nx.single_source_shortest_path_length(reverse_graph, root)
                if depth_dict:
                    depths.extend(list(depth_dict.values()))
                    print(f"  Root {root}: {len(depth_dict)} terms")
                    break  # Use first successful root
            except:
                continue
    
    # Fallback: use any node with in_degree 0 as root
    if not depths:
        roots = [node for node in graph.nodes() if graph.in_degree(node) == 0]
        if roots:
            root = roots[0]
            try:
                depth_dict = nx.single_source_shortest_path_length(reverse_graph, root)
                depths.extend(list(depth_dict.values()))
                print(f"  Using root {root}: {len(depth_dict)} terms")
            except:
                pass
    
    if depths:
        # Sample if too many points for performance
        if len(depths) > 10000:
            depths = np.random.choice(depths, 10000, replace=False)
            sampling_note = f" (sampled from {len(depths):,})"
        else:
            sampling_note = ""
        
        ax.hist(depths, bins=50, alpha=0.7, color='skyblue', 
                edgecolor='black', linewidth=0.5)
        ax.axvline(np.mean(depths), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(depths):.2f}')
        ax.set_xlabel('Depth from Root', fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.set_title(f'Term Depth Distribution{sampling_note}', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No depth data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Term Depth Distribution', fontsize=14, fontweight='bold')

def plot_degree_distribution_optimized(graph: nx.DiGraph, ax):
    """Optimized degree distribution plot."""
    print("Calculating degree distribution...")
    
    # Sample nodes for large graphs
    if graph.number_of_nodes() > 10000:
        sample_size = min(5000, graph.number_of_nodes())
        sample_nodes = np.random.choice(list(graph.nodes()), sample_size, replace=False)
        in_degrees = [graph.in_degree(node) for node in sample_nodes]
        out_degrees = [graph.out_degree(node) for node in sample_nodes]
        sampling_note = f" (sampled {sample_size:,} nodes)"
    else:
        in_degrees = [d for n, d in graph.in_degree()]
        out_degrees = [d for n, d in graph.out_degree()]
        sampling_note = ""
    
    if in_degrees and out_degrees:
        # Use logarithmic bins for better visualization
        max_degree = max(max(in_degrees), max(out_degrees))
        bins = np.logspace(0, np.log10(max_degree + 1), 30)
        
        ax.hist(in_degrees, bins=bins, alpha=0.7, label='In-degree', 
                color='lightcoral', edgecolor='black', linewidth=0.5)
        ax.hist(out_degrees, bins=bins, alpha=0.7, label='Out-degree',
                color='lightgreen', edgecolor='black', linewidth=0.5)
        ax.set_xlabel('Degree', fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        ax.set_title(f'Degree Distribution{sampling_note}', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(alpha=0.3)
        ax.set_xscale('log')
        ax.set_yscale('log')
    else:
        ax.text(0.5, 0.5, 'No degree data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Degree Distribution', fontsize=14, fontweight='bold')

def plot_hierarchy_stats_optimized(graph: nx.DiGraph, annotations: pd.DataFrame, ax):
    """Optimized hierarchy statistics table."""
    ax.axis('off')
    
    # Use cached stats or calculate quickly
    stats = analyze_go_hierarchy(graph, annotations)
    
    table_data = [
        ['Metric', 'Value'],
        ['Total Terms', f"{stats['num_nodes']:,}"],
        ['Total Relationships', f"{stats['num_edges']:,}"],
        ['Annotated Terms', f"{stats['annotated_terms_in_graph']:,}"],
        ['Missing Terms', f"{stats['annotated_terms_not_in_graph']:,}"],
    ]
    
    if 'term_depth' in stats and 'error' not in stats['term_depth']:
        depth_stats = stats['term_depth']
        table_data.extend([
            ['Mean Depth', f"{depth_stats['mean']:.2f}"],
            ['Max Depth', f"{depth_stats['max']}"],
            ['Depth Std', f"{depth_stats['std']:.2f}"],
            ['Terms Measured', f"{depth_stats['num_measured_terms']:,}"],
        ])
    
    if 'degree_analysis' in stats:
        deg = stats['degree_analysis']
        table_data.extend([
            ['Mean In-degree', f"{deg['in_degree_mean']:.2f}"],
            ['Median In-degree', f"{deg['in_degree_median']:.1f}"],
            ['Max In-degree', f"{deg['in_degree_max']}"],
            ['Mean Out-degree', f"{deg['out_degree_mean']:.2f}"],
        ])
    
    table = ax.table(cellText=table_data, 
                    cellLoc='center',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)  # Slightly smaller font
    table.scale(1, 1.8)   # Slightly smaller scale
    
    # Style header row
    for i in range(len(table_data)):
        for j in range(len(table_data[0])):
            if i == 0:  # Header row
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
    
    ax.set_title('GO Hierarchy Statistics', fontsize=16, fontweight='bold', pad=20)

def plot_annotated_overview(graph: nx.DiGraph, annotations: pd.DataFrame, ax):
    """Fast overview of annotated terms (replaces slow network plot)."""
    annotated_terms = set(annotations['go_term'].unique())
    annotated_in_graph = annotated_terms & set(graph.nodes())
    
    # Get ontology distribution of annotated terms
    if 'ontology' in annotations.columns:
        ont_data = annotations[annotations['go_term'].isin(annotated_in_graph)]
        ont_counts = ont_data['ontology'].value_counts()
        
        colors = {'BPO': '#EF4444', 'MFO': '#3B82F6', 'CCO': '#10B981'}
        color_list = [colors.get(ont, 'gray') for ont in ont_counts.index]
        
        bars = ax.bar(ont_counts.index, ont_counts.values, color=color_list, alpha=0.8)
        ax.set_xlabel('Ontology', fontsize=12)
        ax.set_ylabel('Number of Annotated Terms', fontsize=12)
        ax.set_title(f'Annotated Terms by Ontology\n({len(annotated_in_graph):,} terms in graph)', 
                    fontsize=14, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, count in zip(bars, ont_counts.values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                    f'{count:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    else:
        ax.text(0.5, 0.5, f'{len(annotated_in_graph):,} annotated terms in graph', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Annotated Terms Overview', fontsize=14, fontweight='bold')


In [ ]:
print("Starting optimized analysis...")
stats = analyze_go_hierarchy(go_graph, annotations)
print("\nGenerating optimized visualizations...")
visualize_go_hierarchy(go_graph, annotations, "/kaggle/working/go_hierarchy_analysis.png")

In [ ]:
def find_term_ancestors(graph: nx.DiGraph, term: str) -> Set[str]:
    """Find all ancestor terms of a given GO term."""
    try:
        return nx.ancestors(graph, term)
    except nx.NetworkXError:
        return set()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from typing import Set, List, Dict
import numpy as np
from collections import deque

def visualize_term_ancestors(graph: nx.DiGraph, term: str, 
                           annotations: pd.DataFrame = None,
                           max_depth: int = 5,
                           save_path: str = None):
    """
    Visualize the ancestor hierarchy for a specific GO term - FIXED VERSION.
    
    Args:
        graph: GO ontology graph
        term: GO term to analyze
        annotations: Optional annotations for highlighting
        max_depth: Maximum depth to display
        save_path: Optional path to save figure
    """
    print(f"Finding ancestors for {term}...")
    ancestors = find_term_ancestors(graph, term)
    
    if not ancestors:
        print(f"No ancestors found for term {term}")
        # Still create a minimal visualization
        create_minimal_visualization(term, "No ancestors found", save_path)
        return
    
    print(f"Found {len(ancestors)} ancestors. Creating visualization...")
    
    # Create figure with multiple views
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    # 1. Optimized ancestor tree
    ax1 = axes[0, 0]
    plot_ancestor_tree_fixed(graph, term, ancestors, ax1, max_depth)
    
    # 2. Ancestor statistics
    ax2 = axes[0, 1]
    plot_ancestor_stats_fixed(graph, term, ancestors, annotations, ax2)
    
    # 3. Depth distribution
    ax3 = axes[1, 0]
    plot_ancestor_depth_distribution_fixed(graph, term, ancestors, ax3)
    
    # 4. Ontology breakdown
    ax4 = axes[1, 1]
    plot_ancestor_ontology_breakdown_fixed(ancestors, annotations, ax4)
    
    plt.suptitle(f"Ancestor Analysis for {term}", fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved ancestor visualization to {save_path}")
    
    plt.show()

def create_minimal_visualization(term: str, message: str, save_path: str = None):
    """Create a minimal visualization when no ancestors are found."""
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.text(0.5, 0.5, f"Term: {term}\n\n{message}", 
            ha='center', va='center', transform=ax.transAxes, fontsize=14)
    ax.set_title(f"Ancestor Analysis for {term}", fontsize=16, fontweight='bold')
    ax.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_ancestor_tree_fixed(graph: nx.DiGraph, term: str, ancestors: Set[str], 
                           ax, max_depth: int = 5):
    """Fixed ancestor tree plot."""
    # Create subgraph with term and its ancestors
    all_nodes = ancestors.union({term})
    
    if len(all_nodes) > 100:
        print(f"Large ancestor set ({len(all_nodes)} nodes), sampling for visualization...")
        # Get closest ancestors using BFS
        sampled_nodes = get_closest_ancestors_bfs(graph, term, max_nodes=100)
        subgraph = graph.subgraph(sampled_nodes)
    else:
        subgraph = graph.subgraph(all_nodes)
    
    if subgraph.number_of_nodes() <= 1:
        ax.text(0.5, 0.5, 'No connected ancestors to display', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Ancestor Tree', fontsize=14, fontweight='bold')
        return
    
    # Use spring layout (most reliable)
    try:
        pos = nx.spring_layout(subgraph, k=2, iterations=50, seed=42)
    except:
        # Fallback to random layout
        pos = nx.random_layout(subgraph)
    
    # Calculate node properties
    node_colors = []
    node_sizes = []
    
    for node in subgraph.nodes():
        if node == term:
            node_colors.append('red')
            node_sizes.append(500)
        else:
            node_colors.append('lightblue')
            node_sizes.append(200)
    
    # Draw the graph
    nx.draw_networkx_nodes(subgraph, pos, ax=ax, 
                          node_color=node_colors, node_size=node_sizes,
                          alpha=0.8, edgecolors='black')
    
    # Draw edges if they exist
    if subgraph.number_of_edges() > 0:
        nx.draw_networkx_edges(subgraph, pos, ax=ax,
                              edge_color='gray', arrows=True, arrowsize=15,
                              alpha=0.6, arrowstyle='->')
    
    # Add labels for important nodes only
    labels = {}
    for node in subgraph.nodes():
        if node == term or subgraph.degree(node) <= 2:  # Label term and leaf nodes
            labels[node] = node.split(':')[-1]  # Show only the number part
    
    if labels:
        nx.draw_networkx_labels(subgraph, pos, labels, ax=ax,
                               font_size=8, font_weight='bold')
    
    ax.set_title(f'Ancestor Tree\n{len(subgraph.nodes())} of {len(ancestors) + 1} terms shown', 
                fontsize=14, fontweight='bold')
    ax.axis('off')

def get_closest_ancestors_bfs(graph: nx.DiGraph, term: str, max_nodes: int = 100) -> Set[str]:
    """Get closest ancestors using BFS."""
    visited = {term}
    queue = deque([(term, 0)])  # (node, depth)
    closest_ancestors = set()
    
    while queue and len(visited) < max_nodes:
        current, depth = queue.popleft()
        
        for predecessor in graph.predecessors(current):
            if predecessor not in visited:
                visited.add(predecessor)
                closest_ancestors.add(predecessor)
                if len(visited) < max_nodes:
                    queue.append((predecessor, depth + 1))
    
    return visited

def plot_ancestor_stats_fixed(graph: nx.DiGraph, term: str, ancestors: Set[str],
                            annotations: pd.DataFrame, ax):
    """Fixed ancestor statistics plot."""
    ax.axis('off')
    
    # Basic statistics
    total_ancestors = len(ancestors)
    
    # Check if term exists in graph
    term_in_graph = term in graph
    
    # Find roots in ancestry
    root_terms = {'GO:0008150', 'GO:0003674', 'GO:0005575'}
    roots_in_ancestors = root_terms.intersection(ancestors.union({term}))
    
    # Calculate distances to roots
    distances_to_roots = {}
    if term_in_graph:
        reverse_graph = graph.reverse()
        for root in roots_in_ancestors:
            if root in reverse_graph:
                try:
                    dist = nx.shortest_path_length(reverse_graph, root, term)
                    distances_to_roots[root] = dist
                except nx.NetworkXNoPath:
                    distances_to_roots[root] = 'No path'
                except:
                    distances_to_roots[root] = 'Error'
    
    # Get ontology information
    ontology_info = "Not available"
    if annotations is not None and 'ontology' in annotations.columns:
        # Get ontology of the target term
        term_ontology = annotations[annotations['go_term'] == term]['ontology']
        if not term_ontology.empty:
            target_ontology = term_ontology.iloc[0]
        else:
            target_ontology = 'Unknown'
        
        # Get ontology distribution of ancestors
        ancestor_ontologies = annotations[annotations['go_term'].isin(ancestors)]
        if not ancestor_ontologies.empty:
            ontology_counts = ancestor_ontologies['ontology'].value_counts()
            ontology_info = f"Target: {target_ontology}\n"
            for ont, count in ontology_counts.items():
                ontology_info += f"  {ont}: {count:,}\n"
        else:
            ontology_info = f"Target: {target_ontology}\nNo ancestor ontology data"
    
    # Create statistics table
    stats_data = [
        ['Statistic', 'Value'],
        ['Target Term', term],
        ['Total Ancestors', f'{total_ancestors:,}'],
        ['Term in Graph', 'Yes' if term_in_graph else 'No'],
        ['Roots in Ancestry', f'{len(roots_in_ancestors)}'],
    ]
    
    # Add distance information
    for root, dist in distances_to_roots.items():
        root_name = {
            'GO:0008150': 'Biological Process',
            'GO:0003674': 'Molecular Function', 
            'GO:0005575': 'Cellular Component'
        }.get(root, root)
        stats_data.append([f'Distance to {root_name}', f'{dist}'])
    
    stats_data.extend([
        ['', ''],
        ['Ontology Information', ontology_info]
    ])
    
    # Create table
    table = ax.table(cellText=stats_data, 
                    cellLoc='left',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(8)  # Smaller font to fit more content
    table.scale(1, 1.2)
    
    # Style table
    for i in range(len(stats_data)):
        for j in range(len(stats_data[0])):
            if i == 0:  # Header
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
            elif stats_data[i][0] == 'Target Term':
                table[(i, j)].set_facecolor('#FEF3C7')
    
    ax.set_title('Ancestor Statistics', fontsize=14, fontweight='bold')

def plot_ancestor_depth_distribution_fixed(graph: nx.DiGraph, term: str, 
                                         ancestors: Set[str], ax):
    """Fixed depth distribution plot."""
    if term not in graph:
        ax.text(0.5, 0.5, 'Term not in graph\nCannot calculate depths', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Ancestor Depth Distribution', fontsize=14, fontweight='bold')
        return
    
    # Calculate depths using reverse graph (distance TO term)
    try:
        reverse_graph = graph.reverse()
        depths = {}
        
        for ancestor in ancestors:
            if ancestor in reverse_graph:
                try:
                    depth = nx.shortest_path_length(reverse_graph, ancestor, term)
                    depths[ancestor] = depth
                except nx.NetworkXNoPath:
                    continue
        
        depth_values = list(depths.values())
        
        if depth_values:
            # Create histogram
            n, bins, patches = ax.hist(depth_values, bins=min(20, len(set(depth_values))), 
                                     alpha=0.7, color='lightgreen', 
                                     edgecolor='black', linewidth=0.5)
            
            # Add statistics lines
            ax.axvline(np.mean(depth_values), color='red', linestyle='--',
                      label=f'Mean: {np.mean(depth_values):.1f}')
            ax.axvline(np.median(depth_values), color='orange', linestyle=':',
                      label=f'Median: {np.median(depth_values):.1f}')
            
            ax.set_xlabel('Distance to Target Term', fontsize=12)
            ax.set_ylabel('Number of Ancestors', fontsize=12)
            ax.set_title(f'Ancestor Depth Distribution\n({len(depth_values):,} measurable ancestors)', 
                        fontsize=14, fontweight='bold')
            ax.legend()
            ax.grid(alpha=0.3)
            
            # Add some statistics text
            stats_text = f'Min: {min(depth_values)}\nMax: {max(depth_values)}\nStd: {np.std(depth_values):.1f}'
            ax.text(0.95, 0.95, stats_text, transform=ax.transAxes, 
                   ha='right', va='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax.text(0.5, 0.5, 'No measurable depths\n(ancestors not connected to term)', 
                    ha='center', va='center', transform=ax.transAxes)
            ax.set_title('Ancestor Depth Distribution', fontsize=14, fontweight='bold')
            
    except Exception as e:
        ax.text(0.5, 0.5, f'Error calculating depths:\n{str(e)}', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Ancestor Depth Distribution', fontsize=14, fontweight='bold')

def plot_ancestor_ontology_breakdown_fixed(ancestors: Set[str], 
                                         annotations: pd.DataFrame, ax):
    """Fixed ontology breakdown plot."""
    if annotations is None or 'ontology' not in annotations.columns:
        ax.text(0.5, 0.5, 'No ontology data available', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Ontology Breakdown', fontsize=14, fontweight='bold')
        return
    
    # Get ontology distribution of ancestors
    ancestor_ontologies = annotations[annotations['go_term'].isin(ancestors)]
    
    if ancestor_ontologies.empty:
        ax.text(0.5, 0.5, 'No ontology data\nfor ancestors', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Ontology Breakdown', fontsize=14, fontweight='bold')
        return
    
    ontology_counts = ancestor_ontologies['ontology'].value_counts()
    
    # Colors for ontologies
    colors = {'BPO': '#EF4444', 'MFO': '#3B82F6', 'CCO': '#10B981'}
    color_list = [colors.get(ont, 'gray') for ont in ontology_counts.index]
    
    # Create bar chart
    bars = ax.bar(range(len(ontology_counts)), ontology_counts.values,
                 color=color_list, alpha=0.8, edgecolor='black')
    
    ax.set_xticks(range(len(ontology_counts)))
    ax.set_xticklabels([f"{ont}\n({count:,})" for ont, count in ontology_counts.items()])
    ax.set_ylabel('Number of Ancestors', fontsize=12)
    ax.set_title(f'Ancestors by Ontology\n({len(ancestor_ontologies):,} of {len(ancestors):,} ancestors)', 
                fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, count in zip(bars, ontology_counts.values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height,
                f'{count:,}', ha='center', va='bottom', fontweight='bold')


In [ ]:
print("Ancestor visualizations...")

# Single term analysis (fast)
visualize_term_ancestors(
    graph=go_graph,
    term="GO:0006915",
    annotations=annotations,
    save_path="/kaggle/working/term_ancestors.png"
)

In [ ]:
def quick_ancestor_analysis(graph: nx.DiGraph, terms: List[str], 
                          annotations: pd.DataFrame = None,
                          save_path: str = None):
    """
    Quick analysis of ancestors for multiple terms - FIXED VERSION.
    
    Args:
        graph: GO ontology graph
        terms: List of GO terms to analyze
        annotations: Optional annotations DataFrame
    """
    print("Quick Ancestor Analysis")
    print("=" * 50)
    
    results = []
    for term in terms:
        ancestors = find_term_ancestors(graph, term)
        
        # Get ontology if available
        if annotations is not None and 'ontology' in annotations.columns:
            term_ont = annotations[annotations['go_term'] == term]['ontology']
            ontology = term_ont.iloc[0] if not term_ont.empty else 'Unknown'
        else:
            ontology = 'Unknown'
        
        # FIXED: Check if term's hierarchy connects to roots
        roots_reached = check_roots_connected(graph, term)
        
        results.append({
            'term': term,
            'ontology': ontology,
            'ancestor_count': len(ancestors),
            'roots': roots_reached
        })
    
    # Create summary visualization
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Ancestor counts
    terms_list = [f"{r['term']}\n({r['ontology']})" for r in results]
    counts = [r['ancestor_count'] for r in results]
    
    bars = ax1.bar(terms_list, counts, 
                  color=['#EF4444', '#3B82F6', '#10B981'][:len(terms)],
                  alpha=0.8, edgecolor='black')
    ax1.set_ylabel('Number of Ancestors')
    ax1.set_title('Ancestor Counts by Term', fontsize=14, fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, count in zip(bars, counts):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{count:,}', ha='center', va='bottom', fontweight='bold')
    
    # 2. Roots reached - FIXED
    roots_data = [r['roots'] for r in results]
    root_names = ['BP', 'MF', 'CC']  # Simplified root names
    
    # Create stacked bar chart showing which roots are reached
    bottom = np.zeros(len(results))
    colors = ['#EF4444', '#3B82F6', '#10B981']  # BPO, MFO, CCO colors
    
    for i in range(3):  # For each possible root
        root_counts = [1 if r['roots'] > i else 0 for r in results]
        ax2.bar(terms_list, root_counts, bottom=bottom, 
                color=colors[i], alpha=0.8, edgecolor='black', 
                label=root_names[i])
        bottom += root_counts
    
    ax2.set_ylabel('Roots Reached')
    ax2.set_title('Ontology Roots Reached', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    ax2.legend(title='Roots')
    ax2.set_ylim(0, 3.5)
    
    # 3. Depth analysis
    depth_data = []
    for term in terms:
        try:
            # Calculate average depth from roots
            avg_depth = calculate_average_depth(graph, term)
            depth_data.append(avg_depth if avg_depth is not None else 0)
        except:
            depth_data.append(0)
    
    bars = ax3.bar(terms_list, depth_data, 
                  color='lightgreen', alpha=0.8, edgecolor='black')
    ax3.set_ylabel('Average Depth from Roots')
    ax3.set_title('Hierarchical Depth', fontsize=14, fontweight='bold')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, depth in zip(bars, depth_data):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{depth:.1f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved comparison visualization to {save_path}")
    plt.show()
    
    # Print detailed summary
    print("\nDetailed Summary:")
    print("-" * 60)
    for result in results:
        print(f"Term: {result['term']} ({result['ontology']})")
        print(f"  Ancestors: {result['ancestor_count']:,}")
        print(f"  Roots reached: {result['roots']}/3")
        print(f"  Root details: {get_root_details(graph, result['term'])}")
        print()

def check_roots_connected(graph: nx.DiGraph, term: str) -> int:
    """
    Check how many of the three main roots are connected to the term.
    
    Returns:
        Number of roots (0-3) that are connected to the term
    """
    root_terms = {
        'GO:0008150',  # biological_process
        'GO:0003674',  # molecular_function  
        'GO:0005575',  # cellular_component
    }
    
    roots_connected = 0
    reverse_graph = graph.reverse()
    
    for root in root_terms:
        if root in reverse_graph:
            try:
                # Check if there's a path from root to term
                if nx.has_path(reverse_graph, root, term):
                    roots_connected += 1
            except:
                continue
    
    return roots_connected

def get_root_details(graph: nx.DiGraph, term: str) -> str:
    """Get detailed information about which roots are connected."""
    root_mapping = {
        'GO:0008150': 'BP',
        'GO:0003674': 'MF',
        'GO:0005575': 'CC'
    }
    
    connected_roots = []
    reverse_graph = graph.reverse()
    
    for root, short_name in root_mapping.items():
        if root in reverse_graph:
            try:
                if nx.has_path(reverse_graph, root, term):
                    connected_roots.append(short_name)
            except:
                pass
    
    return ", ".join(connected_roots) if connected_roots else "None"

def calculate_average_depth(graph: nx.DiGraph, term: str) -> float:
    """Calculate average depth of term from connected roots."""
    root_terms = ['GO:0008150', 'GO:0003674', 'GO:0005575']
    depths = []
    reverse_graph = graph.reverse()
    
    for root in root_terms:
        if root in reverse_graph:
            try:
                depth = nx.shortest_path_length(reverse_graph, root, term)
                depths.append(depth)
            except:
                continue
    
    return np.mean(depths) if depths else None

# Additional diagnostic function
def diagnose_term_roots(graph: nx.DiGraph, term: str):
    """Diagnose root connectivity for a single term."""
    print(f"\nDiagnosing root connectivity for {term}:")
    print("-" * 40)
    
    root_terms = {
        'GO:0008150': 'Biological Process',
        'GO:0003674': 'Molecular Function',
        'GO:0005575': 'Cellular Component'
    }
    
    reverse_graph = graph.reverse()
    
    for root, description in root_terms.items():
        print(f"\n{description} ({root}):")
        
        if root not in reverse_graph:
            print("  Root not in graph")
            continue
            
        if term not in reverse_graph:
            print("  Term not in graph")
            continue
            
        try:
            has_path = nx.has_path(reverse_graph, root, term)
            print(f"  Path exists: {has_path}")
            
            if has_path:
                distance = nx.shortest_path_length(reverse_graph, root, term)
                print(f"  Distance: {distance}")
                
                # Get the path
                path = nx.shortest_path(reverse_graph, root, term)
                print(f"  Path length: {len(path)} nodes")
                if len(path) <= 8:  # Show short paths
                    print(f"  Path: {' -> '.join(path[:4])} ... -> {term}")
        except nx.NetworkXNoPath:
            print("  No path exists")
        except Exception as e:
            print(f"  Error: {e}")


In [ ]:
# Test the fixed version with diagnostics
print("Testing root connectivity...")

# First, diagnose each term
test_terms = ["GO:0005515", "GO:0009507", "GO:0000122"]
for term in test_terms:
    diagnose_term_roots(go_graph, term)

# Then run the fixed analysis
quick_ancestor_analysis(
    go_graph, 
    test_terms, 
    annotations=annotations,
    save_path="/kaggle/working/ancestors_comparison.png"
)

In [ ]:
def find_term_descendants(graph: nx.DiGraph, term: str) -> Set[str]:
    """Find all descendant terms of a given GO term."""
    try:
        return nx.descendants(graph, term)
    except nx.NetworkXError:
        return set()

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from typing import Set, List, Dict
import numpy as np
from collections import deque, Counter

# Missing helper functions that need to be defined
def find_term_descendants(graph: nx.DiGraph, term: str) -> Set[str]:
    """Find all descendant terms of a given GO term."""
    try:
        return nx.descendants(graph, term)
    except nx.NetworkXError:
        return set()

def calculate_descendant_depth_stats(graph: nx.DiGraph, term: str, descendants: Set[str]) -> Dict:
    """Calculate depth statistics for descendants."""
    try:
        depths = nx.single_source_shortest_path_length(graph, term)
        descendant_depths = [depth for node, depth in depths.items() if node in descendants]
        
        if descendant_depths:
            return {
                'mean': np.mean(descendant_depths),
                'max': max(descendant_depths),
                'min': min(descendant_depths),
                'std': np.std(descendant_depths),
                'count': len(descendant_depths)
            }
    except:
        pass
    return {}

def calculate_branching_stats(graph: nx.DiGraph, term: str, descendants: Set[str]) -> Dict:
    """Calculate branching statistics for the descendant tree."""
    all_nodes = descendants.union({term})
    subgraph = graph.subgraph(all_nodes)
    
    out_degrees = [subgraph.out_degree(node) for node in all_nodes]
    leaves = sum(1 for degree in out_degrees if degree == 0)
    
    return {
        'mean': np.mean(out_degrees) if out_degrees else 0,
        'max': max(out_degrees) if out_degrees else 0,
        'leaves': leaves,
        'internal_nodes': len(all_nodes) - leaves
    }

def create_minimal_descendant_visualization(term: str, message: str, save_path: str = None):
    """Create a minimal visualization when no descendants are found."""
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.text(0.5, 0.5, f"Term: {term}\n\n{message}", 
            ha='center', va='center', transform=ax.transAxes, fontsize=14)
    ax.set_title(f"Descendant Analysis for {term}", fontsize=16, fontweight='bold')
    ax.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def visualize_term_descendants_optimized(graph: nx.DiGraph, term: str, 
                                       annotations: pd.DataFrame = None,
                                       save_path: str = None):
    """
    Optimized visualization for descendant analysis - only reliable, valuable plots.
    
    Args:
        graph: GO ontology graph
        term: GO term to analyze
        annotations: Optional annotations for highlighting
        save_path: Optional path to save figure
    """
    print(f"Finding descendants for {term}...")
    descendants = find_term_descendants(graph, term)
    
    if not descendants:
        print(f"No descendants found for term {term}")
        create_minimal_descendant_visualization(term, "No descendants found", save_path)
        return
    
    print(f"Found {len(descendants)} descendants. Creating optimized visualization...")
    
    # Create figure with only reliable plots
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # 1. Statistics overview (always reliable)
    ax1 = axes[0, 0]
    plot_descendant_stats_optimized(graph, term, descendants, annotations, ax1)
    
    # 2. Depth distribution (reliable)
    ax2 = axes[0, 1]
    plot_descendant_depth_distribution_optimized(graph, term, descendants, ax2)
    
    # 3. Ontology and annotation analysis (reliable)
    ax3 = axes[1, 0]
    plot_descendant_ontology_analysis(descendants, annotations, ax3)
    
    # 4. Branching analysis (reliable)
    ax4 = axes[1, 1]
    plot_branching_analysis(graph, term, descendants, ax4)
    
    plt.suptitle(f"Descendant Analysis for {term}", fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved optimized descendant visualization to {save_path}")
    
    plt.show()

def plot_descendant_stats_optimized(graph: nx.DiGraph, term: str, descendants: Set[str],
                                  annotations: pd.DataFrame, ax):
    """Optimized statistics plot with only reliable metrics."""
    ax.axis('off')
    
    # Basic statistics (always reliable)
    total_descendants = len(descendants)
    
    # Depth statistics (reliable)
    depth_stats = calculate_descendant_depth_stats(graph, term, descendants)
    
    # Branching statistics (reliable)
    branching_stats = calculate_branching_stats(graph, term, descendants)
    
    # Get reliable ontology information
    ontology_info = get_reliable_ontology_info(descendants, annotations, term)
    
    # Annotation statistics (reliable)
    annotation_stats = get_reliable_annotation_stats(descendants, annotations)
    
    # Create statistics table with only reliable metrics
    stats_data = [
        ['Metric', 'Value'],
        ['Target Term', term],
        ['Total Descendants', f'{total_descendants:,}'],
        ['Max Depth', f"{depth_stats.get('max', 'N/A')}"],
        ['Mean Depth', f"{depth_stats.get('mean', 'N/A'):.1f}"],
        ['Branching Factor', f"{branching_stats.get('mean', 'N/A'):.1f}"],
        ['Leaf Nodes', f"{branching_stats.get('leaves', 'N/A'):,}"],
        ['Internal Nodes', f"{branching_stats.get('internal_nodes', 'N/A'):,}"],
    ]
    
    # Add ontology info if available
    if ontology_info != "Not available":
        stats_data.extend([['', ''], ['Ontology', ontology_info]])
    
    # Add annotation stats if available
    if annotation_stats != "No annotation data":
        stats_data.extend([['', ''], ['Annotations', annotation_stats]])
    
    # Create table
    table = ax.table(cellText=stats_data, 
                    cellLoc='left',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.1)
    
    # Style table
    for i in range(len(stats_data)):
        for j in range(len(stats_data[0])):
            if i == 0:  # Header
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
            elif stats_data[i][0] == 'Target Term':
                table[(i, j)].set_facecolor('#FEF3C7')
    
    ax.set_title('Descendant Statistics', fontsize=14, fontweight='bold')

def plot_descendant_depth_distribution_optimized(graph: nx.DiGraph, term: str, descendants: Set[str], ax):
    """Optimized depth distribution plot."""
    try:
        depths = nx.single_source_shortest_path_length(graph, term)
        descendant_depths = [depth for node, depth in depths.items() if node in descendants and depth > 0]
        
        if descendant_depths and len(set(descendant_depths)) > 1:
            # Create histogram with optimal bin calculation
            unique_depths = len(set(descendant_depths))
            bins = min(15, unique_depths)
            
            n, bins, patches = ax.hist(descendant_depths, bins=bins, 
                                     alpha=0.7, color='#3B82F6', 
                                     edgecolor='black', linewidth=0.5)
            
            # Add statistics
            mean_depth = np.mean(descendant_depths)
            median_depth = np.median(descendant_depths)
            
            ax.axvline(mean_depth, color='red', linestyle='--', linewidth=2,
                      label=f'Mean: {mean_depth:.1f}')
            ax.axvline(median_depth, color='orange', linestyle=':', linewidth=2,
                      label=f'Median: {median_depth:.1f}')
            
            ax.set_xlabel('Distance from Target Term', fontsize=12)
            ax.set_ylabel('Number of Descendants', fontsize=12)
            ax.set_title(f'Depth Distribution\n({len(descendant_depths):,} descendants)', 
                        fontsize=14, fontweight='bold')
            ax.legend()
            ax.grid(alpha=0.3)
            
            # Add statistics box
            stats_text = f'Min: {min(descendant_depths)}\nMax: {max(descendant_depths)}\nStd: {np.std(descendant_depths):.1f}'
            ax.text(0.95, 0.95, stats_text, transform=ax.transAxes, 
                   ha='right', va='top', fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
        else:
            # Handle cases with insufficient depth variation
            if descendant_depths:
                ax.text(0.5, 0.7, f'All descendants at depth: {descendant_depths[0]}', 
                        ha='center', va='center', transform=ax.transAxes, fontsize=12)
            else:
                ax.text(0.5, 0.5, 'No depth data available', 
                        ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title('Depth Distribution', fontsize=14, fontweight='bold')
            ax.grid(alpha=0.3)
            
    except Exception as e:
        ax.text(0.5, 0.5, f'Error calculating depths', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Depth Distribution', fontsize=14, fontweight='bold')
        ax.grid(alpha=0.3)

def plot_descendant_ontology_analysis(descendants: Set[str], annotations: pd.DataFrame, ax):
    """Reliable ontology analysis plot."""
    if annotations is None or 'ontology' not in annotations.columns:
        ax.text(0.5, 0.5, 'No ontology data available', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Ontology Analysis', fontsize=14, fontweight='bold')
        ax.grid(alpha=0.3)
        return
    
    # Get descendant annotations
    descendant_data = annotations[annotations['go_term'].isin(descendants)]
    
    if descendant_data.empty:
        ax.text(0.5, 0.5, 'No ontology data\nfor descendants', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Ontology Analysis', fontsize=14, fontweight='bold')
        ax.grid(alpha=0.3)
        return
    
    # Ontology distribution
    ontology_counts = descendant_data['ontology'].value_counts()
    
    # Colors for ontologies
    colors = {'BPO': '#EF4444', 'MFO': '#3B82F6', 'CCO': '#10B981'}
    color_list = [colors.get(ont, 'gray') for ont in ontology_counts.index]
    
    # Create bar chart
    bars = ax.bar(ontology_counts.index, ontology_counts.values,
                 color=color_list, alpha=0.8, edgecolor='black')
    
    ax.set_ylabel('Number of Terms', fontsize=12)
    ax.set_title(f'Ontology Distribution\n({len(descendant_data):,} annotated descendants)', 
                fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels and percentages
    total = ontology_counts.sum()
    for i, (bar, count) in enumerate(zip(bars, ontology_counts.values)):
        percentage = (count / total) * 100
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{count:,}\n({percentage:.1f}%)', ha='center', va='bottom', 
                fontsize=10, fontweight='bold')

def plot_branching_analysis(graph: nx.DiGraph, term: str, descendants: Set[str], ax):
    """Reliable branching analysis plot."""
    all_nodes = descendants.union({term})
    subgraph = graph.subgraph(all_nodes)
    
    # Calculate out-degrees (branching factors)
    out_degrees = [subgraph.out_degree(node) for node in all_nodes]
    
    if out_degrees:
        # Create histogram of branching factors
        unique_degrees = len(set(out_degrees))
        bins = min(10, unique_degrees)
        
        n, bins, patches = ax.hist(out_degrees, bins=bins, 
                                 alpha=0.7, color='#10B981',
                                 edgecolor='black', linewidth=0.5)
        
        # Calculate statistics
        mean_degree = np.mean(out_degrees)
        max_degree = max(out_degrees)
        leaves = sum(1 for degree in out_degrees if degree == 0)
        leaf_percentage = (leaves / len(out_degrees)) * 100
        
        ax.axvline(mean_degree, color='red', linestyle='--', linewidth=2,
                  label=f'Mean: {mean_degree:.1f}')
        
        ax.set_xlabel('Branching Factor (Out-degree)', fontsize=12)
        ax.set_ylabel('Number of Terms', fontsize=12)
        ax.set_title(f'Branching Analysis\n{leaves:,} leaves ({leaf_percentage:.1f}%)', 
                    fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(alpha=0.3)
        
        # Add statistics
        stats_text = f'Max: {max_degree}\nLeaves: {leaves:,}\nTotal: {len(out_degrees):,}'
        ax.text(0.95, 0.95, stats_text, transform=ax.transAxes, 
               ha='right', va='top', fontsize=10,
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    else:
        ax.text(0.5, 0.5, 'No branching data', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Branching Analysis', fontsize=14, fontweight='bold')
        ax.grid(alpha=0.3)

def get_reliable_ontology_info(descendants: Set[str], annotations: pd.DataFrame, term: str) -> str:
    """Get reliable ontology information."""
    if annotations is None or 'ontology' not in annotations.columns:
        return "Not available"
    
    # Get target term ontology
    term_ontology = "Unknown"
    term_ont_data = annotations[annotations['go_term'] == term]['ontology']
    if not term_ont_data.empty:
        term_ontology = term_ont_data.iloc[0]
    
    # Get descendant ontology distribution
    descendant_ontologies = annotations[annotations['go_term'].isin(descendants)]
    if not descendant_ontologies.empty:
        ontology_counts = descendant_ontologies['ontology'].value_counts()
        total_annotated = len(descendant_ontologies)
        
        info = f"Target: {term_ontology}\n"
        for ont, count in ontology_counts.items():
            percentage = (count / total_annotated) * 100
            info += f"{ont}: {count:,} ({percentage:.1f}%)\n"
        return info.strip()
    
    return f"Target: {term_ontology}\nNo descendant data"

def get_reliable_annotation_stats(descendants: Set[str], annotations: pd.DataFrame) -> str:
    """Get reliable annotation statistics."""
    if annotations is None:
        return "Not available"
    
    descendant_annotations = annotations[annotations['go_term'].isin(descendants)]
    
    if descendant_annotations.empty:
        return "No annotation data"
    
    stats = []
    annotated_terms = descendant_annotations['go_term'].nunique()
    total_annotations = len(descendant_annotations)
    unique_proteins = descendant_annotations['protein_id'].nunique()
    
    stats.append(f"Annotated terms: {annotated_terms:,}")
    stats.append(f"Total annotations: {total_annotations:,}")
    stats.append(f"Unique proteins: {unique_proteins:,}")
    
    # Calculate coverage
    if len(descendants) > 0:
        coverage = (annotated_terms / len(descendants)) * 100
        stats.append(f"Coverage: {coverage:.1f}%")
    
    return "\n".join(stats)

In [ ]:
visualize_term_descendants_optimized(
    graph=go_graph,
    term="GO:0006915", 
    annotations=annotations,
    save_path="/kaggle/working/term_descendants.png"
)

In [ ]:
def compare_term_descendants_optimized(graph: nx.DiGraph, 
                                     terms: List[str], 
                                     annotations: pd.DataFrame = None,
                                     max_terms_display: int = 8,
                                     save_path: str = None) -> pd.DataFrame:
    """
    Optimized comparison of GO term descendants with enhanced visualizations.
    
    Args:
        graph: GO ontology graph
        terms: List of GO terms to compare
        annotations: Optional annotations DataFrame for additional insights
        max_terms_display: Maximum number of terms to display in detail
        save_path: Optional path to save figure
        
    Returns:
        DataFrame with comparison statistics
    """
    print("🔍 OPTIMIZED TERM DESCENDANT COMPARISON")
    print("=" * 60)
    
    # Validate input terms
    valid_terms = [term for term in terms if term in graph]
    if len(valid_terms) != len(terms):
        invalid_terms = set(terms) - set(valid_terms)
        print(f"⚠️  Warning: {len(invalid_terms)} terms not found in graph: {invalid_terms}")
    
    if not valid_terms:
        print("❌ No valid terms to analyze")
        return pd.DataFrame()
    
    # Limit display terms for readability
    display_terms = valid_terms[:max_terms_display]
    if len(valid_terms) > max_terms_display:
        print(f"📊 Displaying first {max_terms_display} of {len(valid_terms)} terms for clarity")
    
    # Collect comprehensive statistics
    results = []
    for term in valid_terms:
        descendants = find_term_descendants(graph, term)
        
        # Get ontology information
        ontology = get_term_ontology(term, annotations)
        
        # Calculate comprehensive statistics
        depth_stats = calculate_descendant_depth_stats(graph, term, descendants)
        branching_stats = calculate_branching_stats(graph, term, descendants)
        connectivity_stats = calculate_connectivity_stats(graph, term, descendants)
        
        # Annotation statistics
        annotation_stats = calculate_annotation_stats(descendants, annotations)
        
        results.append({
            'term': term,
            'ontology': ontology,
            'descendant_count': len(descendants),
            'mean_depth': depth_stats.get('mean', 0),
            'max_depth': depth_stats.get('max', 0),
            'depth_std': depth_stats.get('std', 0),
            'branching_factor': branching_stats.get('mean', 0),
            'max_branching': branching_stats.get('max', 0),
            'leaf_count': branching_stats.get('leaves', 0),
            'internal_nodes': branching_stats.get('internal_nodes', 0),
            'graph_density': connectivity_stats.get('density', 0),
            'annotated_descendants': annotation_stats.get('annotated_count', 0),
            'annotation_coverage': annotation_stats.get('coverage', 0),
            'avg_annotations_per_term': annotation_stats.get('avg_annotations', 0)
        })
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    # Create enhanced visualization
    create_comparison_visualization(results_df, display_terms, save_path)
    
    # Print comprehensive summary
    print_comparison_summary(results_df, valid_terms)
    
    return results_df

def get_term_ontology(term: str, annotations: pd.DataFrame) -> str:
    """Get ontology for a term from annotations."""
    if annotations is None or 'ontology' not in annotations.columns:
        return "Unknown"
    
    term_data = annotations[annotations['go_term'] == term]
    if not term_data.empty:
        return term_data['ontology'].iloc[0]
    return "Unknown"

def calculate_connectivity_stats(graph: nx.DiGraph, term: str, descendants: Set[str]) -> Dict:
    """Calculate connectivity statistics for descendant subgraph."""
    all_nodes = descendants.union({term})
    subgraph = graph.subgraph(all_nodes)
    
    stats = {}
    
    if subgraph.number_of_nodes() > 1:
        stats['density'] = nx.density(subgraph)
        
        # Calculate connectivity measures
        try:
            if nx.is_weakly_connected(subgraph):
                stats['diameter'] = nx.diameter(subgraph)
            else:
                # For disconnected graphs, calculate for largest component
                largest_cc = max(nx.weakly_connected_components(subgraph), key=len)
                if len(largest_cc) > 1:
                    largest_subgraph = subgraph.subgraph(largest_cc)
                    stats['diameter_largest_cc'] = nx.diameter(largest_subgraph)
        except:
            stats['diameter'] = 'N/A'
    
    return stats

def calculate_annotation_stats(descendants: Set[str], annotations: pd.DataFrame) -> Dict:
    """Calculate annotation statistics for descendants."""
    stats = {
        'annotated_count': 0,
        'coverage': 0,
        'avg_annotations': 0
    }
    
    if annotations is None or len(descendants) == 0:
        return stats
    
    descendant_annotations = annotations[annotations['go_term'].isin(descendants)]
    
    if not descendant_annotations.empty:
        annotated_terms = descendant_annotations['go_term'].nunique()
        total_annotations = len(descendant_annotations)
        
        stats['annotated_count'] = annotated_terms
        stats['coverage'] = (annotated_terms / len(descendants)) * 100
        stats['avg_annotations'] = total_annotations / annotated_terms if annotated_terms > 0 else 0
    
    return stats

def create_comparison_visualization(results_df: pd.DataFrame, 
                                  display_terms: List[str], 
                                  save_path: str = None):
    """Create enhanced comparison visualization."""
    if results_df.empty:
        print("❌ No data to visualize")
        return
    
    # Filter to display terms only
    display_df = results_df[results_df['term'].isin(display_terms)].copy()
    
    # Create enhanced visualization layout
    fig = plt.figure(figsize=(20, 16))
    
    # Define grid layout
    gs = fig.add_gridspec(3, 3, height_ratios=[1, 1, 0.8])
    
    # 1. Main comparison metrics
    ax1 = fig.add_subplot(gs[0, 0])
    plot_descendant_comparison(display_df, ax1)
    
    # 2. Depth analysis
    ax2 = fig.add_subplot(gs[0, 1])
    plot_depth_analysis(display_df, ax2)
    
    # 3. Branching complexity
    ax3 = fig.add_subplot(gs[0, 2])
    plot_branching_analysis_comparison(display_df, ax3)
    
    # 4. Structural metrics
    ax4 = fig.add_subplot(gs[1, 0])
    plot_structural_metrics(display_df, ax4)
    
    # 5. Annotation coverage
    ax5 = fig.add_subplot(gs[1, 1])
    plot_annotation_coverage(display_df, ax5)
    
    # 6. Term relationships
    ax6 = fig.add_subplot(gs[1, 2])
    plot_term_relationships(display_df, ax6)
    
    # 7. Comprehensive statistics table
    ax7 = fig.add_subplot(gs[2, :])
    plot_comprehensive_statistics(display_df, ax7)
    
    plt.suptitle('GO Term Descendant Comparison Analysis', fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"💾 Comparison visualization saved to: {save_path}")
    
    plt.show()

def plot_descendant_comparison(df: pd.DataFrame, ax):
    """Plot main descendant comparison with enhanced styling."""
    terms_display = [f"{row['term']}\n({row['ontology']})" for _, row in df.iterrows()]
    counts = df['descendant_count'].values
    
    # Create gradient color based on count
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(counts)))
    
    bars = ax.bar(terms_display, counts, color=colors, alpha=0.8, 
                  edgecolor='black', linewidth=1.5)
    
    ax.set_ylabel('Number of Descendants', fontsize=12, fontweight='bold')
    ax.set_title('Descendant Count Comparison', fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', rotation=45, labelsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Enhanced value labels
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height * 1.02,
                f'{count:,}', ha='center', va='bottom', 
                fontsize=11, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))
    
    # Add ranking
    for i, (bar, count) in enumerate(zip(bars, counts)):
        rank = f"#{i+1}"
        ax.text(bar.get_x() + bar.get_width()/2, -height * 0.1,
                rank, ha='center', va='top', fontsize=9, 
                fontweight='bold', color='red')

def plot_depth_analysis(df: pd.DataFrame, ax):
    """Plot comprehensive depth analysis."""
    terms_display = [f"{row['term']}\n({row['ontology']})" for _, row in df.iterrows()]
    
    x = np.arange(len(terms_display))
    width = 0.35
    
    # Mean and max depth bars
    bars1 = ax.bar(x - width/2, df['mean_depth'], width, 
                   label='Mean Depth', color='#FF6B6B', alpha=0.8)
    bars2 = ax.bar(x + width/2, df['max_depth'], width, 
                   label='Max Depth', color='#4ECDC4', alpha=0.8)
    
    ax.set_ylabel('Depth', fontsize=12, fontweight='bold')
    ax.set_title('Hierarchy Depth Analysis', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(terms_display, rotation=45, ha='right', fontsize=9)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels
    for bars, values in [(bars1, df['mean_depth']), (bars2, df['max_depth'])]:
        for bar, value in zip(bars, values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.02,
                    f'{value:.1f}', ha='center', va='bottom', 
                    fontsize=9, fontweight='bold')

def plot_branching_analysis_comparison(df: pd.DataFrame, ax):
    """Plot branching factor comparison with leaf information."""
    terms_display = [f"{row['term']}\n({row['ontology']})" for _, row in df.iterrows()]
    
    x = np.arange(len(terms_display))
    width = 0.6
    
    # Branching factor bars
    bars = ax.bar(x, df['branching_factor'], width, 
                  color='#45B7D1', alpha=0.8, label='Branching Factor')
    
    ax.set_ylabel('Average Branching Factor', fontsize=12, fontweight='bold')
    ax.set_title('Branching Complexity', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(terms_display, rotation=45, ha='right', fontsize=9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add branching factor values
    for bar, value in zip(bars, df['branching_factor']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.02,
                f'{value:.2f}', ha='center', va='bottom', 
                fontsize=10, fontweight='bold')
    
    # Add leaf percentage as text above bars
    for i, (_, row) in enumerate(df.iterrows()):
        leaf_pct = (row['leaf_count'] / row['descendant_count']) * 100
        ax.text(i, row['branching_factor'] * 1.15,
                f'Leaves: {leaf_pct:.1f}%', ha='center', va='bottom',
                fontsize=8, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='yellow', alpha=0.7))

def plot_structural_metrics(df: pd.DataFrame, ax):
    """Plot structural metrics comparison."""
    metrics = ['Internal Nodes', 'Leaf Nodes']
    x = np.arange(len(df))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, df['internal_nodes'], width, 
                   label='Internal Nodes', color='#96CEB4', alpha=0.8)
    bars2 = ax.bar(x + width/2, df['leaf_count'], width, 
                   label='Leaf Nodes', color='#FFEAA7', alpha=0.8)
    
    ax.set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')
    ax.set_title('Node Type Distribution', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([row['ontology'] for _, row in df.iterrows()], fontsize=10)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add ratio annotations
    for i, (_, row) in enumerate(df.iterrows()):
        total = row['internal_nodes'] + row['leaf_count']
        if total > 0:
            leaf_ratio = (row['leaf_count'] / total) * 100
            ax.text(i, max(row['internal_nodes'], row['leaf_count']) * 1.05,
                    f'Leaf Ratio: {leaf_ratio:.1f}%', ha='center', va='bottom',
                    fontsize=8, fontweight='bold')

def plot_annotation_coverage(df: pd.DataFrame, ax):
    """Plot annotation coverage metrics."""
    if 'annotation_coverage' not in df.columns:
        ax.text(0.5, 0.5, 'No annotation data available', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Annotation Coverage', fontsize=14, fontweight='bold')
        return
    
    terms_display = [f"{row['term']}\n({row['ontology']})" for _, row in df.iterrows()]
    
    # Create stacked bar chart for annotation metrics
    coverage = df['annotation_coverage'].fillna(0)
    missing_coverage = 100 - coverage
    
    bars1 = ax.bar(terms_display, coverage, label='Annotated', 
                   color='#6A0572', alpha=0.8)
    bars2 = ax.bar(terms_display, missing_coverage, bottom=coverage, 
                   label='Not Annotated', color='#AB83A1', alpha=0.5)
    
    ax.set_ylabel('Coverage (%)', fontsize=12, fontweight='bold')
    ax.set_title('Annotation Coverage', fontsize=14, fontweight='bold')
    ax.set_xticklabels(terms_display, rotation=45, ha='right', fontsize=9)
    ax.legend(fontsize=10)
    ax.set_ylim(0, 100)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add coverage percentages
    for bar, cov in zip(bars1, coverage):
        if cov > 0:
            ax.text(bar.get_x() + bar.get_width()/2, cov/2,
                    f'{cov:.1f}%', ha='center', va='center',
                    fontsize=9, fontweight='bold', color='white')

def plot_term_relationships(df: pd.DataFrame, ax):
    """Plot relationships between different metrics."""
    if len(df) < 2:
        ax.text(0.5, 0.5, 'Insufficient data for relationship analysis', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_title('Term Relationships', fontsize=14, fontweight='bold')
        return
    
    # Create scatter plot of descendant count vs branching factor
    scatter = ax.scatter(df['descendant_count'], df['branching_factor'],
                        c=df['mean_depth'], s=df['max_depth']*20, 
                        cmap='viridis', alpha=0.7, edgecolors='black')
    
    # Add term labels
    for _, row in df.iterrows():
        ax.annotate(row['term'], 
                   (row['descendant_count'], row['branching_factor']),
                   xytext=(5, 5), textcoords='offset points', fontsize=8,
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))
    
    ax.set_xlabel('Number of Descendants', fontsize=12, fontweight='bold')
    ax.set_ylabel('Branching Factor', fontsize=12, fontweight='bold')
    ax.set_title('Structural Relationships', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3, linestyle='--')
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Mean Depth', fontsize=10)

def plot_comprehensive_statistics(df: pd.DataFrame, ax):
    """Plot comprehensive statistics table."""
    ax.axis('off')
    
    # Prepare table data
    table_data = [['Term', 'Ontology', 'Descendants', 'Mean Depth', 
                   'Max Depth', 'Branching', 'Leaves', 'Coverage%']]
    
    for _, row in df.iterrows():
        coverage = row.get('annotation_coverage', 0)
        table_data.append([
            row['term'],
            row['ontology'],
            f"{row['descendant_count']:,}",
            f"{row['mean_depth']:.1f}",
            f"{row['max_depth']}",
            f"{row['branching_factor']:.2f}",
            f"{row['leaf_count']:,}",
            f"{coverage:.1f}%" if pd.notna(coverage) else "N/A"
        ])
    
    # Create table
    table = ax.table(cellText=table_data, 
                    cellLoc='center',
                    loc='center',
                    bbox=[0, 0, 1, 1])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.8)
    
    # Style table
    for i in range(len(table_data)):
        for j in range(len(table_data[0])):
            if i == 0:  # Header
                table[(i, j)].set_facecolor('#2E86AB')
                table[(i, j)].set_text_props(weight='bold', color='white', size=10)
            else:
                # Color code by ontology
                ontology = table_data[i][1]
                colors = {'BPO': '#FF6B6B', 'MFO': '#4ECDC4', 'CCO': '#45B7D1', 'Unknown': '#F7F7F7'}
                table[(i, j)].set_facecolor(colors.get(ontology, '#F7F7F7'))

def print_comparison_summary(df: pd.DataFrame, all_terms: List[str]):
    """Print comprehensive comparison summary."""
    print("\n📊 COMPREHENSIVE COMPARISON SUMMARY")
    print("=" * 80)
    
    # Overall statistics
    print(f"\n📈 Overall Statistics:")
    print(f"   • Terms analyzed: {len(df)}")
    print(f"   • Total descendants: {df['descendant_count'].sum():,}")
    print(f"   • Average descendants per term: {df['descendant_count'].mean():.0f}")
    print(f"   • Depth range: {df['mean_depth'].min():.1f} - {df['mean_depth'].max():.1f}")
    
    # Top performers
    max_descendants = df.loc[df['descendant_count'].idxmax()]
    max_depth = df.loc[df['max_depth'].idxmax()]
    max_branching = df.loc[df['branching_factor'].idxmax()]
    
    print(f"\n🏆 Top Performers:")
    print(f"   • Most descendants: {max_descendants['term']} ({max_descendants['descendant_count']:,})")
    print(f"   • Deepest hierarchy: {max_depth['term']} (max depth: {max_depth['max_depth']})")
    print(f"   • Highest branching: {max_branching['term']} ({max_branching['branching_factor']:.2f})")
    
    # Ontology breakdown
    if 'ontology' in df.columns:
        ontology_counts = df['ontology'].value_counts()
        print(f"\n🔬 Ontology Distribution:")
        for ont, count in ontology_counts.items():
            ont_df = df[df['ontology'] == ont]
            avg_descendants = ont_df['descendant_count'].mean()
            print(f"   • {ont}: {count} terms, avg {avg_descendants:.0f} descendants")
    
    # Annotation statistics
    if 'annotation_coverage' in df.columns:
        coverage_stats = df['annotation_coverage'].describe()
        print(f"\n📝 Annotation Coverage:")
        print(f"   • Mean coverage: {coverage_stats['mean']:.1f}%")
        print(f"   • Range: {coverage_stats['min']:.1f}% - {coverage_stats['max']:.1f}%")
    
    print(f"\n{'=' * 80}")

# Quick usage examples:
def demonstrate_comparison_functions(graph: nx.DiGraph, annotations: pd.DataFrame):
    """Demonstrate the optimized comparison function."""
    
    # Example terms from different ontologies
    test_terms = [
        "GO:0008150",  # biological_process
        "GO:0003674",  # molecular_function
        "GO:0005575",  # cellular_component
        "GO:0006915",  # apoptosis
        "GO:0005524",  # ATP binding
        "GO:0005737",  # cytoplasm
    ]
    
    print("🧪 DEMONSTRATING OPTIMIZED TERM COMPARISON")
    print("=" * 60)
    
    # Run comparison
    results = compare_term_descendants_optimized(
        graph=graph,
        terms=test_terms,
        annotations=annotations,
        max_terms_display=6,
        save_path="/kaggle/working/term_comparison_optimized.png"
    )
    
    return results

# Main usage:
# results = compare_term_descendants_optimized(go_graph, terms_list, annotations)

In [ ]:
# Optimized comparison
test_terms = ["GO:0005515", "GO:0006915", "GO:0000122"]
compare_term_descendants_optimized(
    graph=go_graph,
    terms=test_terms,
    annotations=annotations,
    save_path="/kaggle/working/descendant_comparison_optimized.png"
)

# Taxonomy Analysis Functions

In [ ]:
def analyze_taxonomy_distribution(taxonomy: pd.DataFrame) -> Dict:
    """
    Analyze distribution of proteins across species.
    
    Args:
        taxonomy: DataFrame with protein_id and taxon_id
        
    Returns:
        Dictionary with taxonomy statistics
    """
    stats = {}
    
    taxon_counts = taxonomy['taxon_id'].value_counts()
    
    stats['num_species'] = len(taxon_counts)
    stats['num_proteins'] = len(taxonomy)
    stats['proteins_per_species'] = {
        'mean': taxon_counts.mean(),
        'median': taxon_counts.median(),
        'min': taxon_counts.min(),
        'max': taxon_counts.max()
    }
    
    # Top species
    stats['top_species'] = taxon_counts.head(10).to_dict()
    
    print("\nTaxonomy Distribution:")
    print(f"Total species: {stats['num_species']}")
    print(f"Total proteins: {stats['num_proteins']:,}")
    print(f"Proteins per species - Mean: {stats['proteins_per_species']['mean']:.1f}, "
          f"Median: {stats['proteins_per_species']['median']:.1f}")
    
    print("\nTop 10 Species by Protein Count:")
    for taxon_id, count in list(stats['top_species'].items())[:10]:
        print(f"  Taxon {taxon_id}: {count:,} proteins")
    
    return stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict

def visualize_taxonomy_distribution(taxonomy: pd.DataFrame, 
                                  top_n: int = 20,
                                  save_path: str = None) -> Dict:
    """
    Visualize distribution of proteins across species with comprehensive plots.
    
    Args:
        taxonomy: DataFrame with protein_id and taxon_id
        top_n: Number of top species to display in detail
        save_path: Optional path to save figure
        
    Returns:
        Dictionary with taxonomy statistics
    """
    # Get statistics from the analysis function
    stats = analyze_taxonomy_distribution(taxonomy)
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. Top species bar chart
    ax1 = axes[0, 0]
    plot_top_species(stats, top_n, ax1)
    
    # 2. Species distribution histogram
    ax2 = axes[0, 1]
    plot_species_distribution(taxonomy, ax2)
    
    # 3. Cumulative distribution
    ax3 = axes[0, 2]
    plot_cumulative_distribution(taxonomy, ax3)
    
    # 4. Statistics overview
    ax4 = axes[1, 0]
    plot_taxonomy_stats(stats, ax4)
    
    # 5. Protein coverage by species
    ax5 = axes[1, 1]
    plot_protein_coverage(taxonomy, ax5)
    
    # 6. Species rank plot
    ax6 = axes[1, 2]
    plot_species_rank(taxonomy, ax6)
    
    plt.suptitle('Taxonomic Distribution Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved taxonomy visualization to {save_path}")
    
    plt.show()
    
    return stats

def plot_top_species(stats: Dict, top_n: int, ax):
    """Plot top N species by protein count."""
    top_species = dict(list(stats['top_species'].items())[:top_n])
    
    if not top_species:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'Top {top_n} Species', fontsize=14, fontweight='bold')
        return
    
    species_ids = list(top_species.keys())
    counts = list(top_species.values())
    
    # Create bar chart
    bars = ax.bar(range(len(species_ids)), counts, 
                 color=plt.cm.viridis(np.linspace(0, 1, len(species_ids))),
                 alpha=0.8, edgecolor='black')
    
    ax.set_xlabel('Species (Taxon ID)', fontsize=12)
    ax.set_ylabel('Number of Proteins', fontsize=12)
    ax.set_title(f'Top {top_n} Species by Protein Count', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(species_ids)))
    ax.set_xticklabels(species_ids, rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height,
                f'{count:,}', ha='center', va='bottom', 
                fontsize=9, fontweight='bold')
    
    # Add percentage labels
    total_proteins = stats['num_proteins']
    for i, (bar, count) in enumerate(zip(bars, counts)):
        percentage = (count / total_proteins) * 100
        ax.text(bar.get_x() + bar.get_width()/2, height/2,
                f'{percentage:.1f}%', ha='center', va='center',
                fontsize=8, color='white', fontweight='bold')

def plot_species_distribution(taxonomy: pd.DataFrame, ax):
    """Plot distribution of proteins per species."""
    taxon_counts = taxonomy['taxon_id'].value_counts()
    
    if len(taxon_counts) == 0:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Species Distribution', fontsize=14, fontweight='bold')
        return
    
    # Create histogram
    counts = taxon_counts.values
    
    # Use log scale for better visualization of long-tailed distributions
    if counts.max() / counts.min() > 100:  # Large dynamic range
        bins = np.logspace(np.log10(counts.min()), np.log10(counts.max()), 30)
        ax.set_xscale('log')
        xlabel = 'Proteins per Species (log scale)'
    else:
        bins = 30
        xlabel = 'Proteins per Species'
    
    n, bins, patches = ax.hist(counts, bins=bins, alpha=0.7, 
                              color='skyblue', edgecolor='black')
    
    # Add statistics lines
    mean_count = counts.mean()
    median_count = np.median(counts)
    
    ax.axvline(mean_count, color='red', linestyle='--', linewidth=2,
              label=f'Mean: {mean_count:.1f}')
    ax.axvline(median_count, color='orange', linestyle=':', linewidth=2,
              label=f'Median: {median_count:.1f}')
    
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel('Number of Species', fontsize=12)
    ax.set_title('Distribution of Proteins per Species', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Add statistics box
    stats_text = f'Total species: {len(counts):,}\nMin: {counts.min():,}\nMax: {counts.max():,}'
    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
           ha='right', va='top', fontsize=10,
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

def plot_cumulative_distribution(taxonomy: pd.DataFrame, ax):
    """Plot cumulative distribution of proteins across species."""
    taxon_counts = taxonomy['taxon_id'].value_counts()
    
    if len(taxon_counts) == 0:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Cumulative Distribution', fontsize=14, fontweight='bold')
        return
    
    # Calculate cumulative distribution
    sorted_counts = np.sort(taxon_counts.values)[::-1]  # Descending
    cumulative = np.cumsum(sorted_counts)
    cumulative_percentage = (cumulative / cumulative[-1]) * 100
    species_percentage = (np.arange(len(sorted_counts)) + 1) / len(sorted_counts) * 100
    
    # Plot cumulative distribution
    ax.plot(species_percentage, cumulative_percentage, 
           linewidth=3, color='purple', alpha=0.8)
    ax.fill_between(species_percentage, cumulative_percentage, alpha=0.3, color='purple')
    
    # Add reference lines
    ax.axhline(80, color='red', linestyle='--', alpha=0.7, label='80% of proteins')
    ax.axvline(20, color='orange', linestyle=':', alpha=0.7, label='20% of species')
    
    # Find intersection points
    idx_80_proteins = np.argmax(cumulative_percentage >= 80)
    idx_20_species = int(0.2 * len(species_percentage))
    
    if idx_80_proteins < len(species_percentage):
        ax.plot(species_percentage[idx_80_proteins], 80, 'ro', markersize=8)
        ax.text(species_percentage[idx_80_proteins], 85, 
               f'{species_percentage[idx_80_proteins]:.1f}% species\nhave 80% proteins',
               ha='center', va='bottom', fontsize=9)
    
    ax.set_xlabel('Percentage of Species (%)', fontsize=12)
    ax.set_ylabel('Cumulative Percentage of Proteins (%)', fontsize=12)
    ax.set_title('Cumulative Distribution of Proteins', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

def plot_taxonomy_stats(stats: Dict, ax):
    """Plot taxonomy statistics overview."""
    ax.axis('off')
    
    # Create comprehensive statistics table
    stats_data = [
        ['Metric', 'Value'],
        ['Total Species', f"{stats['num_species']:,}"],
        ['Total Proteins', f"{stats['num_proteins']:,}"],
        ['Mean Proteins/Species', f"{stats['proteins_per_species']['mean']:.1f}"],
        ['Median Proteins/Species', f"{stats['proteins_per_species']['median']:.1f}"],
        ['Min Proteins/Species', f"{stats['proteins_per_species']['min']:,}"],
        ['Max Proteins/Species', f"{stats['proteins_per_species']['max']:,}"],
    ]
    
    # Add coverage statistics
    taxon_counts = list(stats['top_species'].values())
    if taxon_counts:
        top_10_coverage = sum(list(stats['top_species'].values())[:10]) / stats['num_proteins'] * 100
        top_20_coverage = sum(list(stats['top_species'].values())[:20]) / stats['num_proteins'] * 100
        
        stats_data.extend([
            ['', ''],
            ['Coverage Statistics', ''],
            ['Top 10 Species Coverage', f'{top_10_coverage:.1f}%'],
            ['Top 20 Species Coverage', f'{top_20_coverage:.1f}%'],
        ])
    
    # Create table
    table = ax.table(cellText=stats_data, 
                    cellLoc='center',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.2)
    
    # Style table
    for i in range(len(stats_data)):
        for j in range(len(stats_data[0])):
            if i == 0:  # Header row
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
            elif i >= len(stats_data) - 4:  # Coverage statistics
                table[(i, j)].set_facecolor('#E5E7EB')
    
    ax.set_title('Taxonomy Statistics Overview', fontsize=14, fontweight='bold')

def plot_protein_coverage(taxonomy: pd.DataFrame, ax):
    """Plot protein coverage by species rank."""
    taxon_counts = taxonomy['taxon_id'].value_counts()
    
    if len(taxon_counts) == 0:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Protein Coverage', fontsize=14, fontweight='bold')
        return
    
    sorted_counts = np.sort(taxon_counts.values)[::-1]
    cumulative_coverage = np.cumsum(sorted_counts) / np.sum(sorted_counts) * 100
    
    # Create area plot
    species_ranks = np.arange(1, len(sorted_counts) + 1)
    
    ax.fill_between(species_ranks, cumulative_coverage, alpha=0.6, color='green')
    ax.plot(species_ranks, cumulative_coverage, linewidth=2, color='darkgreen')
    
    # Add reference lines and annotations
    for coverage in [50, 80, 90, 95]:
        idx = np.argmax(cumulative_coverage >= coverage)
        if idx < len(species_ranks):
            ax.axvline(species_ranks[idx], color='red', linestyle='--', alpha=0.5)
            ax.text(species_ranks[idx], coverage + 2, 
                   f'{species_ranks[idx]} species\n({coverage}% coverage)',
                   ha='center', va='bottom', fontsize=8,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel('Species Rank', fontsize=12)
    ax.set_ylabel('Cumulative Protein Coverage (%)', fontsize=12)
    ax.set_title('Protein Coverage by Species Rank', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.set_xscale('log')  # Log scale for better visualization

def plot_species_rank(taxonomy: pd.DataFrame, ax):
    """Plot species rank vs protein count (Zipf-like distribution)."""
    taxon_counts = taxonomy['taxon_id'].value_counts()
    
    if len(taxon_counts) == 0:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Species Rank Distribution', fontsize=14, fontweight='bold')
        return
    
    sorted_counts = np.sort(taxon_counts.values)[::-1]
    ranks = np.arange(1, len(sorted_counts) + 1)
    
    # Create rank plot
    ax.loglog(ranks, sorted_counts, 'o-', alpha=0.7, markersize=4, linewidth=1)
    
    # Add power law reference line if applicable
    if len(sorted_counts) > 10:
        try:
            # Fit a simple power law for reference
            log_ranks = np.log(ranks[:100])  # Use first 100 points for fit
            log_counts = np.log(sorted_counts[:100])
            slope, intercept = np.polyfit(log_ranks, log_counts, 1)
            
            # Plot fitted line
            fitted_counts = np.exp(intercept) * ranks**slope
            ax.loglog(ranks, fitted_counts, 'r--', alpha=0.8, 
                     label=f'Power law fit (α={-slope:.2f})')
            ax.legend()
        except:
            pass
    
    ax.set_xlabel('Species Rank (log scale)', fontsize=12)
    ax.set_ylabel('Protein Count (log scale)', fontsize=12)
    ax.set_title('Species Rank vs Protein Count', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3, which='both')

# Simplified version for quick analysis
def quick_taxonomy_visualization(taxonomy: pd.DataFrame, 
                               save_path: str = None) -> Dict:
    """
    Quick taxonomy visualization with essential plots only.
    
    Args:
        taxonomy: DataFrame with protein_id and taxon_id
        save_path: Optional path to save figure
        
    Returns:
        Dictionary with taxonomy statistics
    """
    stats = analyze_taxonomy_distribution(taxonomy)
    
    # Create simplified visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Top species
    plot_top_species(stats, 15, axes[0])
    
    # 2. Distribution
    plot_species_distribution(taxonomy, axes[1])
    
    plt.suptitle('Taxonomic Distribution Overview', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved quick taxonomy visualization to {save_path}")
    
    plt.show()
    
    return stats

# Enhanced analysis function with visualization integration
def analyze_taxonomy_distribution_enhanced(taxonomy: pd.DataFrame, 
                                         visualize: bool = True,
                                         save_path: str = None) -> Dict:
    """
    Enhanced taxonomy analysis with optional visualization.
    
    Args:
        taxonomy: DataFrame with protein_id and taxon_id
        visualize: Whether to create visualizations
        save_path: Optional path to save figure
        
    Returns:
        Dictionary with taxonomy statistics
    """
    
    if visualize:
        if len(taxonomy) > 1000:  # Large dataset - use quick visualization
            quick_taxonomy_visualization(taxonomy, save_path)
        else:  # Small dataset - use comprehensive visualization
            visualize_taxonomy_distribution(taxonomy, save_path=save_path)
    
    # return stats

In [ ]:
analyze_taxonomy_distribution_enhanced(taxonomy, visualize=True, save_path="/kaggle/working/taxonomy_distribution.png")

In [ ]:
def analyze_annotations_by_species(annotations: pd.DataFrame, taxonomy: pd.DataFrame) -> pd.DataFrame:
    """
    Analyze annotation patterns across species.
    
    Returns:
        DataFrame with per-species annotation statistics
    """
    # Merge annotations with taxonomy
    merged = annotations.merge(taxonomy, on='protein_id', how='left')
    
    # Group by species
    species_stats = merged.groupby('taxon_id').agg({
        'protein_id': 'nunique',
        'go_term': 'count'
    }).reset_index()
    
    species_stats.columns = ['taxon_id', 'num_proteins', 'num_annotations']
    species_stats['annotations_per_protein'] = (
        species_stats['num_annotations'] / species_stats['num_proteins']
    )
    
    species_stats = species_stats.sort_values('num_proteins', ascending=False)
    
    return species_stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List
import pandas as pd

def visualize_annotations_by_species(annotations: pd.DataFrame, 
                                   taxonomy: pd.DataFrame,
                                   top_n: int = 20,
                                   save_path: str = None) -> pd.DataFrame:
    """
    Visualize annotation patterns across species with comprehensive plots.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology
        taxonomy: DataFrame with protein_id and taxon_id
        top_n: Number of top species to display in detail
        save_path: Optional path to save figure
        
    Returns:
        DataFrame with per-species annotation statistics
    """
    # Get statistics from the analysis function
    species_stats = analyze_annotations_by_species(annotations, taxonomy)
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(22, 14))
    
    # 1. Top species by protein count
    ax1 = axes[0, 0]
    plot_species_protein_counts(species_stats, top_n, ax1)
    
    # 2. Top species by annotation count
    ax2 = axes[0, 1]
    plot_species_annotation_counts(species_stats, top_n, ax2)
    
    # 3. Annotations per protein distribution
    ax3 = axes[0, 2]
    plot_annotations_per_protein(species_stats, ax3)
    
    # 4. Correlation analysis
    ax4 = axes[1, 0]
    plot_correlation_analysis(species_stats, ax4)
    
    # 5. Species statistics overview
    ax5 = axes[1, 1]
    plot_species_statistics(species_stats, ax5)
    
    # 6. Annotation density analysis
    ax6 = axes[1, 2]
    plot_annotation_density(annotations, taxonomy, species_stats, ax6)
    
    plt.suptitle('Annotation Patterns Across Species', fontsize=18, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved species annotation visualization to {save_path}")
    
    plt.show()
    
    return species_stats

def plot_species_protein_counts(species_stats: pd.DataFrame, top_n: int, ax):
    """Plot top N species by protein count."""
    top_species = species_stats.head(top_n)
    
    if top_species.empty:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'Top {top_n} Species by Protein Count', fontsize=14, fontweight='bold')
        return
    
    species_ids = top_species['taxon_id'].astype(str)
    protein_counts = top_species['num_proteins']
    
    # Create horizontal bar chart for better readability
    bars = ax.barh(range(len(species_ids)), protein_counts,
                  color=plt.cm.plasma(np.linspace(0, 1, len(species_ids))),
                  alpha=0.8, edgecolor='black')
    
    ax.set_xlabel('Number of Proteins', fontsize=12)
    ax.set_ylabel('Species (Taxon ID)', fontsize=12)
    ax.set_title(f'Top {top_n} Species by Protein Count', fontsize=14, fontweight='bold')
    ax.set_yticks(range(len(species_ids)))
    ax.set_yticklabels(species_ids, fontsize=9)
    ax.grid(axis='x', alpha=0.3)
    ax.invert_yaxis()  # Highest count at top
    
    # Add value labels on bars
    for i, (bar, count) in enumerate(zip(bars, protein_counts)):
        width = bar.get_width()
        ax.text(width + width * 0.01, bar.get_y() + bar.get_height()/2,
                f'{count:,}', ha='left', va='center', 
                fontsize=9, fontweight='bold')
    
    # Add percentage of total
    total_proteins = species_stats['num_proteins'].sum()
    for i, (bar, count) in enumerate(zip(bars, protein_counts)):
        percentage = (count / total_proteins) * 100
        ax.text(width * 0.3, bar.get_y() + bar.get_height()/2,
                f'{percentage:.1f}%', ha='center', va='center',
                fontsize=8, color='white', fontweight='bold')

def plot_species_annotation_counts(species_stats: pd.DataFrame, top_n: int, ax):
    """Plot top N species by annotation count."""
    top_species = species_stats.head(top_n)
    
    if top_species.empty:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'Top {top_n} Species by Annotation Count', fontsize=14, fontweight='bold')
        return
    
    species_ids = top_species['taxon_id'].astype(str)
    annotation_counts = top_species['num_annotations']
    
    # Create bar chart
    bars = ax.bar(range(len(species_ids)), annotation_counts,
                 color=plt.cm.viridis(np.linspace(0, 1, len(species_ids))),
                 alpha=0.8, edgecolor='black')
    
    ax.set_xlabel('Species (Taxon ID)', fontsize=12)
    ax.set_ylabel('Number of Annotations', fontsize=12)
    ax.set_title(f'Top {top_n} Species by Annotation Count', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(species_ids)))
    ax.set_xticklabels(species_ids, rotation=45, ha='right', fontsize=9)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, count in zip(bars, annotation_counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height,
                f'{count:,}', ha='center', va='bottom', 
                fontsize=9, fontweight='bold')
    
    # Add annotation density
    for i, (bar, count, proteins) in enumerate(zip(bars, annotation_counts, top_species['num_proteins'])):
        density = count / proteins
        ax.text(bar.get_x() + bar.get_width()/2, height * 0.3,
                f'{density:.1f} annot/prot', ha='center', va='center',
                fontsize=7, color='white', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

def plot_annotations_per_protein(species_stats: pd.DataFrame, ax):
    """Plot distribution of annotations per protein across species."""
    if species_stats.empty:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Annotations per Protein Distribution', fontsize=14, fontweight='bold')
        return
    
    annotations_per_protein = species_stats['annotations_per_protein']
    
    # Create histogram
    n, bins, patches = ax.hist(annotations_per_protein, bins=30, 
                              alpha=0.7, color='lightcoral',
                              edgecolor='black', linewidth=0.5)
    
    # Add statistics lines
    mean_density = annotations_per_protein.mean()
    median_density = annotations_per_protein.median()
    
    ax.axvline(mean_density, color='red', linestyle='--', linewidth=2,
              label=f'Mean: {mean_density:.2f}')
    ax.axvline(median_density, color='orange', linestyle=':', linewidth=2,
              label=f'Median: {median_density:.2f}')
    
    ax.set_xlabel('Annotations per Protein', fontsize=12)
    ax.set_ylabel('Number of Species', fontsize=12)
    ax.set_title('Distribution of Annotation Density Across Species', 
                fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Add statistics box
    stats_text = (f'Species: {len(annotations_per_protein):,}\n'
                 f'Min: {annotations_per_protein.min():.2f}\n'
                 f'Max: {annotations_per_protein.max():.2f}\n'
                 f'Std: {annotations_per_protein.std():.2f}')
    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
           ha='right', va='top', fontsize=10,
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

def plot_correlation_analysis(species_stats: pd.DataFrame, ax):
    """Plot correlation between protein count and annotation count."""
    if len(species_stats) < 2:
        ax.text(0.5, 0.5, 'Insufficient data for correlation analysis', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Protein vs Annotation Correlation', fontsize=14, fontweight='bold')
        return
    
    # Create scatter plot
    scatter = ax.scatter(species_stats['num_proteins'], 
                        species_stats['num_annotations'],
                        c=species_stats['annotations_per_protein'],
                        cmap='viridis', alpha=0.7, s=50,
                        edgecolors='black', linewidth=0.5)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Annotations per Protein', fontsize=10)
    
    # Calculate correlation
    correlation = species_stats['num_proteins'].corr(species_stats['num_annotations'])
    
    # Add trend line
    if not species_stats['num_proteins'].isna().any() and not species_stats['num_annotations'].isna().any():
        z = np.polyfit(species_stats['num_proteins'], species_stats['num_annotations'], 1)
        p = np.poly1d(z)
        ax.plot(species_stats['num_proteins'], p(species_stats['num_proteins']), 
               "r--", alpha=0.8, linewidth=2,
               label=f'Correlation: {correlation:.3f}')
    
    ax.set_xlabel('Number of Proteins', fontsize=12)
    ax.set_ylabel('Number of Annotations', fontsize=12)
    ax.set_title(f'Protein vs Annotation Correlation\n(r = {correlation:.3f})', 
                fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Use log scale if data range is large
    if (species_stats['num_proteins'].max() / species_stats['num_proteins'].min() > 100 or
        species_stats['num_annotations'].max() / species_stats['num_annotations'].min() > 100):
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('Number of Proteins (log scale)')
        ax.set_ylabel('Number of Annotations (log scale)')

def plot_species_statistics(species_stats: pd.DataFrame, ax):
    """Plot comprehensive species statistics overview."""
    ax.axis('off')
    
    if species_stats.empty:
        ax.text(0.5, 0.5, 'No species statistics available', 
                ha='center', va='center', transform=ax.transAxes)
        return
    
    # Calculate comprehensive statistics
    total_species = len(species_stats)
    total_proteins = species_stats['num_proteins'].sum()
    total_annotations = species_stats['num_annotations'].sum()
    
    # Coverage statistics
    top_10_protein_coverage = species_stats.head(10)['num_proteins'].sum() / total_proteins * 100
    top_10_annotation_coverage = species_stats.head(10)['num_annotations'].sum() / total_annotations * 100
    
    # Density statistics
    density_stats = species_stats['annotations_per_protein'].describe()
    
    # Create statistics table
    stats_data = [
        ['Metric', 'Value'],
        ['Total Species', f'{total_species:,}'],
        ['Total Proteins', f'{total_proteins:,}'],
        ['Total Annotations', f'{total_annotations:,}'],
        ['', ''],
        ['Coverage Statistics', ''],
        ['Top 10 Species - Protein Coverage', f'{top_10_protein_coverage:.1f}%'],
        ['Top 10 Species - Annotation Coverage', f'{top_10_annotation_coverage:.1f}%'],
        ['', ''],
        ['Annotation Density', ''],
        ['Mean Annotations/Protein', f'{density_stats["mean"]:.2f}'],
        ['Median Annotations/Protein', f'{density_stats["50%"]:.2f}'],
        ['Min Annotations/Protein', f'{density_stats["min"]:.2f}'],
        ['Max Annotations/Protein', f'{density_stats["max"]:.2f}'],
    ]
    
    # Create table
    table = ax.table(cellText=stats_data, 
                    cellLoc='left',
                    loc='center',
                    bbox=[0.1, 0.1, 0.8, 0.8])
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.1)
    
    # Style table
    for i in range(len(stats_data)):
        for j in range(len(stats_data[0])):
            if i == 0:  # Header row
                table[(i, j)].set_facecolor('#4B5563')
                table[(i, j)].set_text_props(weight='bold', color='white')
            elif i in [5, 9]:  # Section headers
                table[(i, j)].set_facecolor('#E5E7EB')
                table[(i, j)].set_text_props(weight='bold')
    
    ax.set_title('Species Annotation Statistics Overview', fontsize=14, fontweight='bold')

def plot_annotation_density(annotations: pd.DataFrame, taxonomy: pd.DataFrame, 
                          species_stats: pd.DataFrame, ax):
    """Plot annotation density analysis across species."""
    if species_stats.empty:
        ax.text(0.5, 0.5, 'No species data available', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Annotation Density Analysis', fontsize=14, fontweight='bold')
        return
    
    # Calculate density percentiles
    density_percentiles = np.percentile(species_stats['annotations_per_protein'], 
                                       [25, 50, 75, 90, 95])
    
    # Create density categories
    low_density = species_stats[species_stats['annotations_per_protein'] <= density_percentiles[0]]
    medium_density = species_stats[(species_stats['annotations_per_protein'] > density_percentiles[0]) & 
                                 (species_stats['annotations_per_protein'] <= density_percentiles[2])]
    high_density = species_stats[species_stats['annotations_per_protein'] > density_percentiles[2]]
    
    # Create stacked bar chart
    categories = ['Low Density', 'Medium Density', 'High Density']
    counts = [len(low_density), len(medium_density), len(high_density)]
    colors = ['#EF4444', '#F59E0B', '#10B981']
    
    bars = ax.bar(categories, counts, color=colors, alpha=0.8, edgecolor='black')
    
    ax.set_ylabel('Number of Species', fontsize=12)
    ax.set_title('Species by Annotation Density Category', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels and percentages
    total_species = len(species_stats)
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        percentage = (count / total_species) * 100
        ax.text(bar.get_x() + bar.get_width()/2, height,
                f'{count:,}\n({percentage:.1f}%)', ha='center', va='bottom',
                fontsize=10, fontweight='bold')
    
    # Add density range annotations
    ax.text(0.02, 0.98, f'Density Ranges:\n'
                       f'Low: ≤{density_percentiles[0]:.2f}\n'
                       f'Medium: {density_percentiles[0]:.2f}-{density_percentiles[2]:.2f}\n'
                       f'High: >{density_percentiles[2]:.2f}',
            transform=ax.transAxes, ha='left', va='top', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Quick visualization version
def quick_species_annotation_visualization(annotations: pd.DataFrame,
                                         taxonomy: pd.DataFrame,
                                         top_n: int = 15,
                                         save_path: str = None) -> pd.DataFrame:
    """
    Quick visualization of species annotation patterns.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology
        taxonomy: DataFrame with protein_id and taxon_id
        top_n: Number of top species to display
        save_path: Optional path to save figure
        
    Returns:
        DataFrame with per-species annotation statistics
    """
    species_stats = analyze_annotations_by_species(annotations, taxonomy)
    
    # Create simplified visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Top species by protein count
    plot_species_protein_counts(species_stats, top_n, axes[0])
    
    # 2. Correlation analysis
    plot_correlation_analysis(species_stats, axes[1])
    
    plt.suptitle('Species Annotation Patterns - Quick Overview', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved quick species annotation visualization to {save_path}")
    
    plt.show()
    
    return species_stats

# Enhanced analysis with visualization
def analyze_annotations_by_species_enhanced(annotations: pd.DataFrame,
                                          taxonomy: pd.DataFrame,
                                          visualize: bool = True,
                                          top_n: int = 20,
                                          save_path: str = None) -> pd.DataFrame:
    """
    Enhanced species annotation analysis with optional visualization.
    
    Args:
        annotations: DataFrame with protein_id, go_term, ontology
        taxonomy: DataFrame with protein_id and taxon_id
        visualize: Whether to create visualizations
        top_n: Number of top species to display
        save_path: Optional path to save figure
        
    Returns:
        DataFrame with per-species annotation statistics
    """
    species_stats = analyze_annotations_by_species(annotations, taxonomy)
    
    if visualize:
        if len(species_stats) > 50:  # Large dataset - use quick visualization
            quick_species_annotation_visualization(annotations, taxonomy, top_n, save_path)
        else:  # Small dataset - use comprehensive visualization
            visualize_annotations_by_species(annotations, taxonomy, top_n, save_path)
    
    return species_stats

In [ ]:
analyze_annotations_by_species_enhanced(annotations, taxonomy, visualize=True, save_path="/kaggle/working/annotations_by_species.png")